In [ ]:
import os

ret = os.path.exists('./RuleBasedTesting')
print("Exists:", ret)
if (ret == False):
  !git clone https://github.com/safednn-nasa/RuleBasedTesting.git

!pwd
!ls
os.chdir('./RuleBasedTesting')
!pwd
!ls
!unzip input_files.zip


Exists: True
/content
RuleBasedTesting  sample_data
/content/RuleBasedTesting
input_files.zip  README.md	   test_X_MNIST.txt
mnist0_adv.h5	 RuleBasedTesting  test_Y_MNIST.txt
Archive:  input_files.zip
   creating: input_files/
  inflating: input_files/mnist_test_eps0.1.txt  
  inflating: input_files/train_X_MNIST.txt  
  inflating: input_files/train_Y_MNIST.txt  


In [ ]:
!pip install tf-keras-vis

import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import time
#from Models.loss import smoothL1
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Input, InputLayer
from tensorflow.keras.layers import Activation, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2

from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam import GradcamPlusPlus

from tf_keras_vis.utils import normalize

import matplotlib.pyplot as plt



import math
import io
import os
from collections import namedtuple
import sys

import operator

import copy

#!pwd
#os.chdir("./drive/MyDrive/Colab Notebooks")
#!pwd
#!ls

print(tf.__version__)

from platform import python_version

print(python_version())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.9.2
3.8.16


In [ ]:
import os
import keras
import argparse
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
#from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler
from keras.models import Model, load_model
from keras import optimizers, regularizers
from keras import backend as K
import pickle

def get_prediction(img, tensor = None):
  
    if tensor == "ALL":
        full_list = []
        for tensor in ['dense_1', 'dense_2', 'dense_3']:
            layer_name = tensor           
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
           # print(outputs)
            layer_outputs_test = []
            fingerprint1 = ((outputs[0] > 0.0).astype('int'))
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result[0])
            full_list.append(layer_outputs_test) 
            
        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)     
    
        return layer_outputs_test

def get_prediction_vals(img, tensor = None):
  
    if tensor == "ALL":
        full_list = []
        for tensor in ['activation_1','activation_2','activation_3']:
            layer_name = tensor           
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
            op = np.array(outputs)
            #print(op.shape)
            flat_op = op.flatten()
            #print(flat_op.shape)
            fingerprint1 = list(flat_op)
            #print(layer.name,fingerprint1)
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result)
            full_list.append(layer_outputs_test) 
            
        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        #fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        fingerprint1 = outputs[0]
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)     
    
        return layer_outputs_test





def fingerprint_suffix(img,ten):
    return (get_prediction(img, tensor=ten)>0.0).astype('int')

def fingerprint_suffix_vals(img,ten):
    return get_prediction_vals(img, tensor=ten)

#from tf_keras_vis.gradcam import Gradcam
#from tf_keras_vis.gradcam import GradcamPlusPlus

def model_modifier_act1_layer(current_model):
    target_layer = current_model.get_layer(name='activation_1') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act2_layer(current_model):
    target_layer = current_model.get_layer(name='activation_2') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act3_layer(current_model):
    target_layer = current_model.get_layer(name='activation_3') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_last_layer(current_model):
    target_layer = current_model.get_layer(name='activation_5') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer(current_model):
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer_inps(current_model):
    src_layer = current_model.get_layer(name='activation')
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=src_layer.input,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def loss_gen_sum(node_list):
    def loss(output):
        print(output.shape)
        op = np.array(output)

        #if (op.ndim > 2):
        op = op.flatten()
        output = np.empty((1,len(op)))
        print(output.shape)
        output[0] = op
        
        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss


def loss_gen_sum1(node_list):
    def loss(output):
        
        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss

def loss_gen_sep(node):
    def loss(output):
        loss_val = output[0][node]
        return loss_val
    return loss


#import matplotlib.pyplot as plt
#from keras.models import load_model
print('Loading the model:')
model=load_model('./mnist0_adv.h5') 
print("Printing summary of the model:")
model.summary()








Loading the model:
Printing summary of the model:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 2)         20        
                                                                 
 activation_1 (Activation)   (None, 26, 26, 2)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 4)         76        
                                                                 
 activation_2 (Activation)   (None, 24, 24, 4)         0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 576)               0         
    

In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = False,ALL = False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;
  
   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = int(neuron_ids[ind])
       vsig = float(neuron_sig[ind])
       val = layer_vals[v]
       #print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found 

In [ ]:
import numpy as np
import csv

num_classes = 10

a=[]
path = 'input_files/train_Y_MNIST.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c = int(row[0])
    a.append(c)
train_Y = np.array(a)
print("TRAIN IDEAL LABELS:", train_Y.shape)

gen_data = []
gen_labels = []
ind = 0
a=[]
path = 'input_files/train_X_MNIST.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c=[]
    for i in range(0,len(row)):
     #   c.append(float(row[i])/255.0)
        c.append(float(row[i])/255.0)
    c = np.reshape(c, (28,28))
    a.append(c)
    #if ((ind % 10) == 0):
    #  gen_data.append(c)
    #  gen_labels.append(train_Y[ind])
    #ind = ind + 1
train_X = np.array(a)
print("TRAIN CLEAN DATA:", train_X.shape)

train_labels = (model.predict(train_X)).argmax(axis=1)
print("ACTUAL Train LABELS:", train_labels.shape)
PASS = 0
FAIL = 0
cor_ind = 0
for indx in range(0, len(train_labels)):
  if (train_labels[indx] == train_Y[indx]):
    PASS = PASS + 1
    if ((ind % 5) == 0):
      gen_data.append(train_X[ind])
      gen_labels.append(train_Y[indx])
      cor_ind = cor_ind + 1
  else:
    FAIL = FAIL + 1
    gen_data.append(train_X[ind])
    gen_labels.append(train_Y[indx])
  ind = ind + 1

print("PASS:", PASS, ",FAIL:", FAIL, ",Train ACCURACY:", (PASS/(PASS+FAIL))*100.0, ", COR IND:", cor_ind)



a=[]
path = 'test_Y_MNIST.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c = int(row[0])
    a.append(c)
testFULL_Y = np.array(a)
print("TEST IDEAL LABELS:", testFULL_Y.shape)


ind = 0
a=[]
path = 'test_X_MNIST.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c=[]
    for i in range(0,len(row)):
     #   c.append(float(row[i])/255.0)
        c.append(float(row[i])/255.0)
    c = np.reshape(c, (28,28))
    a.append(c)
    ind = ind + 1
testFULL_X = np.array(a)
print("TEST CLEAN DATA:", testFULL_X.shape)


test_labels = (model.predict(testFULL_X)).argmax(axis=1)
print("ACTUAL TEST LABELS:", test_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels)):
  if (test_labels[indx] == testFULL_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST FULL ACCURACY:", (PASS/(PASS+FAIL))*100.0)

### RANDOM SPLIT INTO VAL AND TEST SET 
import random
length = (len(testFULL_X))
ind = 0
valinds = []
while (ind < (length)/2):
  ix = random.randint(0,length)
  while(ix in valinds):
    ix = random.randint(0,length)
  valinds.append(ix)
  ind = ind + 1

test_X = []
test_Y = []
val_X = []
val_Y = []
for indx in range(0,length):
  if (indx in valinds):
    val_X.append(testFULL_X[indx])
    val_Y.append(testFULL_Y[indx])
  else:
    test_X.append(testFULL_X[indx])
    test_Y.append(testFULL_Y[indx])

val_X = np.array(val_X)
val_Y = np.array(val_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)
print("VALIDATION SET:", len(val_X), val_X.shape, len(val_Y), val_Y.shape)
print("TEST SET:", len(test_X), test_X.shape, len(test_Y), test_Y.shape)

val_labels = (model.predict(val_X)).argmax(axis=1)
print("ACTUAL Val LABELS:", val_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(val_labels)):
  if (val_labels[indx] == val_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",VAL ACCURACY:", (PASS/(PASS+FAIL))*100.0)

test_labels = (model.predict(test_X)).argmax(axis=1)
print("ACTUAL TEST LABELS:", test_labels.shape)
PASS = 0
FAIL_test = 0
for indx in range(0, len(test_labels)):
  if (test_labels[indx] == test_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL_test = FAIL_test + 1
print("PASS:", PASS, ",FAIL:", FAIL_test, ",TEST ACCURACY:", (PASS/(PASS+FAIL_test))*100.0)

gen_data = np.array(gen_data)
gen_labels = np.array(gen_labels)
gen_act_labels = (model.predict(gen_data)).argmax(axis=1)
print("ACTUAL GEN LABELS:", gen_act_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(gen_labels)):
  if (gen_act_labels[indx] == gen_labels[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",GEN ACCURACY:", (PASS/(PASS+FAIL))*100.0)



# label0_tots = len(np.where(gen_labels == 0 and gen_act_labels == 0)[0])
# label1_tots = len(np.where(gen_labels == 1 and gen_act_labels == 1)[0])
# label2_tots = len(np.where(gen_labels == 2 and gen_act_labels == 2)[0])
# label3_tots = len(np.where(gen_labels == 3 and gen_act_labels == 3)[0])
# label4_tots = len(np.where(gen_labels == 4 and gen_act_labels == 4)[0])
# label5_tots = len(np.where(gen_labels == 5 and gen_act_labels == 5)[0])
# label6_tots = len(np.where(gen_labels == 6 and gen_act_labels == 6)[0])
# label7_tots = len(np.where(gen_labels == 7 and gen_act_labels == 7)[0])
# label8_tots = len(np.where(gen_labels == 8 and gen_act_labels == 8)[0])
# label9_tots = len(np.where(gen_labels == 9 and gen_act_labels == 9)[0])

# print(label0_tots,label1_tots,label2_tots,label3_tots,label4_tots,label5_tots)
# print(label6_tots,label7_tots,label8_tots,label9_tots)
mis_tots = FAIL
print(mis_tots)

print('Get Adv Test Data:')

a=[]
path = 'input_files/mnist_test_eps0.1.txt'
ind = 0
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c=[]
    for i in range(0,len(row)):
     #   c.append(float(row[i])/255.0)
        c.append(float(row[i]))
    c = np.reshape(c, (28,28))
    a.append(c)
    ind = ind + 1
    # if (ind == 5000):
    #   break
test_X_adv_2 = np.array(a)
print("ADV 2 DATA:", test_X_adv_2.shape)


test_labels_adv_2 = (model.predict(test_X_adv_2)).argmax(axis=1)
print("ADV 2 TEST LABELS:", test_labels_adv_2.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_adv_2)):
  if (test_labels_adv_2[indx] == testFULL_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)


#### 2 TEST-SUITEs ( 1 WITH ONLY FROM PASSING TESTS, 1 with passing and failing)
import random
length = len(test_X)
ind = 0
pass_inds = []
while (ind < 2*FAIL_test):
  ix = random.randint(0,length-1)
  while(ix in pass_inds):
    ix = random.randint(0,length-1)
  if (test_labels[ix] == test_Y[ix]):
    pass_inds.append(ix)
    ind = ind + 1

test_pass_X = []
test_pass_Y = []
for indx in range(0,length):
  if (indx in pass_inds):
    test_pass_X.append(test_X[indx])
    test_pass_Y.append(test_Y[indx])


test_pass_X = np.array(test_pass_X)
test_pass_Y = np.array(test_pass_Y)
print("TEST PASS SET:", len(test_pass_X), test_pass_X.shape, len(test_pass_Y), test_pass_Y.shape)

test_labels_pass = (model.predict(test_pass_X)).argmax(axis=1)
print("TEST PASS ACTUAL LABELS:", test_labels_pass.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_pass)):
  if (test_labels_pass[indx] == test_pass_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)


import random
length = len(test_X)
ind = 0
pass_fail_inds = []
ind_pass = 0
ind_fail = 0
while (ind < 2*FAIL_test):
  ix = random.randint(0,length-1)
  while(ix in pass_fail_inds):
    ix = random.randint(0,length-1)
  if (test_labels[ix] == test_Y[ix]):
    if (ind_pass > FAIL_test):
      continue
    else:
      pass_fail_inds.append(ix)
      ind_pass = ind_pass + 1
  else:
    if (ind_fail > FAIL_test):
      continue
    else:
      pass_fail_inds.append(ix)
      ind_fail = ind_fail + 1
  ind = ind + 1

test_pass_fail_X = []
test_pass_fail_Y = []
for indx in range(0,length):
  if (indx in pass_fail_inds):
    test_pass_fail_X.append(test_X[indx])
    test_pass_fail_Y.append(test_Y[indx])


test_pass_fail_X = np.array(test_pass_fail_X)
test_pass_fail_Y = np.array(test_pass_fail_Y)
print("TEST PASS and FAIL SET:", len(test_pass_fail_X), test_pass_fail_X.shape, len(test_pass_fail_Y), test_pass_fail_Y.shape)

test_labels_pass_fail = (model.predict(test_pass_fail_X)).argmax(axis=1)
print("TEST PASS and FAIL ACTUAL LABELS:", test_labels_pass_fail.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_pass_fail)):
  if (test_labels_pass_fail[indx] == test_pass_fail_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

import random
length = len(test_X)
ind = 0
tst_adv_inds = []
while (ind < 2500):
  ix = random.randint(0,length-1)
  while(ix in tst_adv_inds):
    ix = random.randint(0,length-1)
  tst_adv_inds.append(ix)
  ind = ind + 1

length = len(test_X_adv_2)
ind = 0
adv_inds = []
while (ind < 2500):
  ix = random.randint(0,length-1)
  while(ix in adv_inds):
    ix = random.randint(0,length-1)
  adv_inds.append(ix)
  ind = ind + 1

test_adv_X = []
test_adv_Y = []
for indx in range(0,len(test_X)):
  if (indx in tst_adv_inds):
    test_adv_X.append(test_X[indx])
    test_adv_Y.append(test_Y[indx])

for indx in range(0,len(test_X_adv_2)):
  if (indx in adv_inds):
    test_adv_X.append(test_X_adv_2[indx])
    test_adv_Y.append(testFULL_Y[indx])


test_adv_X = np.array(test_adv_X)
test_adv_Y = np.array(test_adv_Y)
print("TEST ADV SET:", len(test_adv_X), test_adv_X.shape, len(test_adv_Y), test_adv_Y.shape)

test_labels_adv = (model.predict(test_adv_X)).argmax(axis=1)
print("TEST ADV ACTUAL LABELS:", test_labels_adv.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_adv)):
  if (test_labels_adv[indx] == test_adv_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

###################### TEST SUITES FOR FUNCTIONAL DIVERSITY
test_biased_X = []
test_biased_Y = []
for indx in range(0, len(test_Y)):
  if ((test_Y[indx] == 0 ) or (test_Y[indx] == 9 )):
    test_biased_X.append(test_X[indx])
    test_biased_Y.append(test_Y[indx])

length = len(test_X)
ind = 0
img_inds = []
while (ind < len(test_biased_X)):
  ix = random.randint(0,length-1)
  while(ix in img_inds):
    ix = random.randint(0,length-1)
  img_inds.append(ix)
  ind = ind + 1

test_unbiased_X = []
test_unbiased_Y = []
for indx in range(0,len(test_X)):
  if (indx in img_inds):
    test_unbiased_X.append(test_X[indx])
    test_unbiased_Y.append(test_Y[indx])

test_biased_X = np.array(test_biased_X)
test_labels_biased = (model.predict(test_biased_X)).argmax(axis=1)
print("TEST BIASED TO LABEL 0 AND 9  LABELS:", test_labels_biased.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_biased)):
  if (test_labels_biased[indx] == test_biased_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

test_unbiased_X = np.array(test_unbiased_X)
test_labels_unbiased = (model.predict(test_unbiased_X)).argmax(axis=1)
print("TEST unbiased LABELS:", test_labels_unbiased.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_unbiased)):
  if (test_labels_unbiased[indx] == test_unbiased_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)







TRAIN IDEAL LABELS: (60000,)
TRAIN CLEAN DATA: (60000, 28, 28)
1875/1875 [==============================] - 12s 6ms/step
ACTUAL Train LABELS: (60000,)
PASS: 59204 ,FAIL: 796 ,Train ACCURACY: 98.67333333333333 , COR IND: 11855
TEST IDEAL LABELS: (10000,)
TEST CLEAN DATA: (10000, 28, 28)
313/313 [==============================] - 2s 5ms/step
ACTUAL TEST LABELS: (10000,)
PASS: 9787 ,FAIL: 213 ,TEST FULL ACCURACY: 97.87
VALIDATION SET: 4999 (4999, 28, 28) 4999 (4999,)
TEST SET: 5001 (5001, 28, 28) 5001 (5001,)
157/157 [==============================] - 1s 5ms/step
ACTUAL Val LABELS: (4999,)
PASS: 4904 ,FAIL: 95 ,VAL ACCURACY: 98.09961992398479
157/157 [==============================] - 1s 5ms/step
ACTUAL TEST LABELS: (5001,)
PASS: 4883 ,FAIL: 118 ,TEST ACCURACY: 97.64047190561888
396/396 [==============================] - 2s 5ms/step
ACTUAL GEN LABELS: (12651,)
PASS: 11855 ,FAIL: 796 ,GEN ACCURACY: 93.7080072721524
796
Get Adv Test Data:
ADV 2 DATA: (10000, 28, 28)
313/313 [===============

In [ ]:
#layer_preds
layer = model.get_layer('dense_1') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(gen_data)
fingerprint_1 = ((outputs[0] > 0.0).astype('int'))
fingerprint1 = outputs[0]

layer = model.get_layer('activation_3') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(gen_data)
fingerprint_2 = ((outputs[0] > 0.0).astype('int'))
fingerprint2 = outputs[0]



PASS = 0
FAIL = 0
val_vec1 = []
val_vec_labels1 = []
actuals_gen = (model.predict(gen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(fingerprint1)):  
  #print(actuals_gen[i],gen_labels[i])
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    val_vec_labels1.append(gen_labels[i])
    val_vec1.append(np.asarray((fingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    #val_vec_labels1.append(10 + gen_labels[i])
    val_vec_labels1.append(1000)
    val_vec1.append(np.asarray((fingerprint1[i]).flatten()))

PASS = 0
FAIL = 0
act_vec1 = []
act_vec_labels1 = []
print("ACT from layer:")
for i in range(0,len(fingerprint_1)):  
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    act_vec_labels1.append(gen_labels[i])
    act_vec1.append(np.asarray((fingerprint_1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    #act_vec_labels1.append(10 + gen_labels[i])
    act_vec_labels1.append(1000)
    act_vec1.append(np.asarray((fingerprint_1[i]).flatten()))

PASS = 0
FAIL = 0
val_vec2 = []
val_vec_labels2 = []
actuals_gen = (model.predict(gen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(fingerprint2)):  
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    val_vec_labels2.append(gen_labels[i])
    val_vec2.append(np.asarray((fingerprint2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    #val_vec_labels2.append(10 + gen_labels[i])
    val_vec_labels2.append(1000)
    val_vec2.append(np.asarray((fingerprint2[i]).flatten()))

PASS = 0
FAIL = 0
act_vec2 = []
act_vec_labels2 = []
print("ACT from layer:")
for i in range(0,len(fingerprint_2)):  
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    act_vec_labels2.append(gen_labels[i])
    act_vec2.append(np.asarray((fingerprint_2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    #act_vec_labels2.append(10 + gen_labels[i])
    act_vec_labels2.append(1000)
    act_vec2.append(np.asarray((fingerprint_2[i]).flatten()))

print("PASS:", PASS, ",FAIL:", FAIL,",ACCURACY:", (PASS/(PASS+FAIL))*100.0)

396/396 [==============================] - 3s 8ms/step
VALS from layer:
ACT from layer:
396/396 [==============================] - 3s 8ms/step
VALS from layer:
ACT from layer:
PASS: 11855 ,FAIL: 796 ,ACCURACY: 93.7080072721524


In [ ]:
from sklearn import tree

val_vec1 = np.asarray(val_vec1)
val_vec_labels1 = np.asarray(val_vec_labels1)
print(val_vec1.shape, val_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator1 = tree.DecisionTreeClassifier()
basic_estimator1.fit(val_vec1, val_vec_labels1)

act_vec1 = np.asarray(act_vec1)
act_vec_labels1 = np.asarray(act_vec_labels1)
print(act_vec1.shape, act_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator1a = tree.DecisionTreeClassifier()
basic_estimator1a.fit(act_vec1, act_vec_labels1)

val_vec2 = np.asarray(val_vec2)
val_vec_labels2 = np.asarray(val_vec_labels2)
print(val_vec2.shape, val_vec_labels2.shape)
basic_estimator2 = tree.DecisionTreeClassifier()
basic_estimator2.fit(val_vec2, val_vec_labels2)

act_vec2 = np.asarray(act_vec2)
act_vec_labels2 = np.asarray(act_vec_labels2)
print(act_vec2.shape, act_vec_labels2.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator2a = tree.DecisionTreeClassifier()
basic_estimator2a.fit(act_vec2, act_vec_labels2)


(12651, 128) (12651,)
(12651, 128) (12651,)
(12651, 128) (12651,)
(12651, 128) (12651,)


DecisionTreeClassifier()

In [ ]:
from tqdm import tqdm
import operator
import pandas as pd

train_suffixes = act_vec1
train_labels = act_vec_labels1

def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig,suffixes=train_suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True



def get_invariant_inp(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - inp: reference input, shape <784,>
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist_inp_images[ref_id]
  ref_suffix = suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,suffixes)
  imgs = []
  cnt = 0
  for indx1 in range(0,len(cluster)):
    img = mnist.train.images(cluster[indx1])
    fnd = 1
    for i in range(0,len(img)):
      if (ref_img[i] != img[i]):
        fnd = 0
        break
    if (fnd == 1):
        ref_id = cnt
    cnt = cnt + 1
    imgs.append(img)
    
  imgs_suffixes = fingerprint_signature(imgs,t_fc2)
  ref_suffix = imgs_suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,imgs_suffixes)
    
  return cluster, neuron_ids, neuron_sig

def get_invariant(estimator, ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_suffix = train_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig


def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]
  
  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
   # paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
   # paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    paths = [[[feature[node]] + p[0],['<='] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0],['>'] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def describe_cluster(cluster, neuron_ids, show_samples=False):
  neuron_sig = train_suffixes[cluster[0]][neuron_ids]
  print("Num neurons in invariant", len(neuron_ids))
  print("Neuron id and signature")
  
  for i in range(0,len(neuron_ids)):
    print("id:", neuron_ids[i], "sig:", neuron_sig[i])
  
  print("Cluster size: ", len(cluster))
  print("Num misclassified", len([i for i in cluster if is_misclassified(i)]))
  if show_samples:
    for i in range(10):
      images = []
      for j in range(10):
        if 10*i + j >= len(cluster):
          break
        images.append(mnist_to_pil_img(mnist.train.images[cluster[10*i+j]]))
      if len(images) > 0:
        show_img(combine(images))
  
def describe_invariants_all_labels_col(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  Minvs_sel_neus = []
  Minvs_sel_sig = []
  Minvs_sel_supp = []
  invs_sel_supp = []
  invs_sel_cl = []
  Minvs_sel_cl = []

  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []

  for cl, invs in all_invariants.items():
   
    for indx in range (0, len(invs)):
      inv = invs[indx]
      

     # print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])

    # MISCLASSIFICATION PATTTERN FOR IDEAL LABELs
      if (cl < 0):
          Minvs_sel_cl.append(cl) 
          Minvs_sel_neus.append(inv[0]) 
          Minvs_sel_sig.append(inv[1])
          Minvs_sel_supp.append(inv[2])

      # MISCLASSIFICATION PATTTERNs FOR ACTUAL LABELS
      if (cl == 1000):
        invs_sel_neus.append(inv[0])
        invs_sel_sig.append(inv[1])
        invs_sel_supp.append(inv[2])
        invs_sel_cl.append(cl)
         
      
      #if ((cl >= 0) ): #and (cl not in corr_invs_sel_labs)):
      if ((cl >= 0) and (cl < 10)): 
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])
      
      

  return (Minvs_sel_neus, Minvs_sel_sig, Minvs_sel_supp, Minvs_sel_cl, invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)


def describe_invariants_all_labels(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  invs_sel_supp = []
  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []
  mis_invs_sel_labs = []
  mis_invs_sel_neus = []
  mis_invs_sel_sig = []
  mis_invs_sel_supp = []

 # print("PRINTING PURE RULES WITH SUPPORT MORE THAN 100 FOR EVERY LABEL:");
  supp = 0
  for cl, invs in all_invariants.items():
   
    for indx in range (0, len(invs)):
      inv = invs[indx]
     # if (inv[2] <= 10):
     #   continue
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
     # print("Class:", cl, "Support:",inv[2])
    
      
      if ((cl >= 0) and (cl < 10)):
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])
      
      if ((cl >= 10) ):
          mis_invs_sel_labs.append(cl)
          mis_invs_sel_neus.append(inv[0])
          mis_invs_sel_sig.append(inv[1])
          mis_invs_sel_supp.append(inv[2])

      continue

      cls = get_suffix_cluster(inv[0],inv[1],suffixes)
      
      neurons = inv[0]
      signature = inv[1]

      if (len(cls) <= 10):
         continue

     # fail_cnt = []
     # fail_label = []
     # pass_cnt = []
     # for ind in range(0,len(cls)):
     #   inp_indx = cls[ind]
     #   if (FP[inp_indx] == 0):
     #     fail_cnt.append(inp_indx)
     #     fail_label.append(IDEAL[inp_indx])
     #   else:
     #     pass_cnt.append(inp_indx)
          
      #fail_perc = (fail_cnt/len(cls))
      #if (inv[2] == 243):
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2],",Pass:",[i for i in pass_cnt],",Mis:",[i for i in fail_cnt] , ",Label:",[i for i in fail_label]);
      
      print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
      #if (indx > 0):
      #  break
     # for ind in range(0, len(cls)):
     #    print(cls[ind])
      
     # break
      if (COMMON == True):
          common_nodes(cls,suffixes)

      if (DEC_PREFX == True):
          decision_prefs(cls,suffixes)

  return (mis_invs_sel_labs, mis_invs_sel_neus, mis_invs_sel_sig, mis_invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)
  
  
def common_nodes(cls,suffixes):
    cnt = 0
    common = np.zeros(10,dtype=int)
    prev = np.zeros(10,dtype=int)
    
    for indx in range(0, len(cls)):
        i = cls[indx]
        cnt = cnt + 1
        for j in range(0,len(suffixes[i])):
          if (common[j] == -1):
             continue
          if ((indx != 0) and (suffixes[i][j] != prev[j])):
             common[j] = -1
          else:
             common[j] = suffixes[i][j]
          prev[j] = suffixes[i][j]


    print('COMMON NODES IN CLUSTER for CLASS:',cl,cnt)
    com = []
    for k in range(0,len(common)):
        if (common[k] != -1):
           com.append((k,common[k]))
    print(com)

    return
    
def decision_prefs(cls,suffixes):
    images = mnist.train.images
    imgsCom = []
    imgs = []
    for indx in range(0, len(cls)):
        print('IMG:')
        print(list(zip(images[cls[indx]])))
        imgs.append(images[cls[indx]])
        imgsCom.append(images[cls[indx]])
            
    dec_prefixes= fingerprint_signature(imgs,layer)
    prefixes = []
    for indx in range(0,len(dec_prefixes)):
       dec_pref = dec_prefixes[indx]
    
       match = 0
       for indx1 in range(0, len(prefixes)):
          match = 1
          for i in range(0,len(prefixes[indx1])):
             if (dec_pref[i] != prefixes[indx1][i]):
                match = 0
                break
          if (match == 1):
             break
    
       if (match == 0):
          prefixes.append(dec_pref)
    
    print('DECISION PREFIXES IN CLUSTER for CLASS:',cl,cnt)
    for k in range(0,len(prefixes)):
      print(prefixes[k])

    return
    

In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)

corr_invs_sel_lay1 = []
corr_invs_sel_lay1a = []
corr_invs_sel_lay2 = []
corr_invs_sel_lay2a = []

mis_invs_sel_lay1 = []
mis_invs_sel_lay1a = []
mis_invs_sel_lay2 = []
mis_invs_sel_lay2a = []


invariants_VAL1 = get_all_invariants(basic_estimator1)
#(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs1, corr_invs_sel_neus1, corr_invs_sel_sig1, corr_invs_sel_supp1) = describe_invariants_all_labels_col(invariants_VAL1)
(mis_invs_sel_labs, mis_invs_sel_neus, mis_invs_sel_sig, mis_invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants_VAL1)

print("\n CORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(corr_invs_sel_neus)):
   corr_invs_sel_lay1.append(1)
   print(corr_invs_sel_labs[indx], corr_invs_sel_neus[indx], corr_invs_sel_sig[indx], corr_invs_sel_supp[indx])

print("\n INCORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(mis_invs_sel_neus)):
   mis_invs_sel_lay1.append(1)
   print(mis_invs_sel_labs[indx], mis_invs_sel_neus[indx], mis_invs_sel_sig[indx], mis_invs_sel_supp[indx])

invariants_ACT1 = get_all_invariants(basic_estimator1a)
#(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs1a, corr_invs_sel_neus1a, corr_invs_sel_sig1a, corr_invs_sel_supp1a) = describe_invariants_all_labels_col(invariants_ACT1)
(mis_invs_sel_labs1, mis_invs_sel_neus1, mis_invs_sel_sig1, mis_invs_sel_supp1, corr_invs_sel_labs1, corr_invs_sel_neus1, corr_invs_sel_sig1, corr_invs_sel_supp1) = describe_invariants_all_labels(invariants_ACT1)

print("\n CORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(corr_invs_sel_neus1)):
   corr_invs_sel_lay1a.append(1)
   print(corr_invs_sel_labs1[indx], corr_invs_sel_neus1[indx], corr_invs_sel_sig1[indx], corr_invs_sel_supp1[indx])

print("\n INCORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(mis_invs_sel_neus1)):
   mis_invs_sel_lay1a.append(1)
   print(mis_invs_sel_labs1[indx], mis_invs_sel_neus1[indx], mis_invs_sel_sig1[indx], mis_invs_sel_supp1[indx])


invariants_VAL2 = get_all_invariants(basic_estimator2)
#(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs2, corr_invs_sel_neus2, corr_invs_sel_sig2, corr_invs_sel_supp2) = describe_invariants_all_labels_col(invariants_VAL2)
(mis_invs_sel_labs2, mis_invs_sel_neus2, mis_invs_sel_sig2, mis_invs_sel_supp2, corr_invs_sel_labs2, corr_invs_sel_neus2, corr_invs_sel_sig2, corr_invs_sel_supp2) = describe_invariants_all_labels(invariants_VAL2)

print("\n CORRECT PATTERNS VAL 2:\n")
for indx in range(0, len(corr_invs_sel_neus2)):
   corr_invs_sel_lay2.append(2)
   print(corr_invs_sel_labs2[indx], corr_invs_sel_neus2[indx], corr_invs_sel_sig2[indx], corr_invs_sel_supp2[indx])

print("\n INCORRECT PATTERNS VAL 2:\n")
for indx in range(0, len(mis_invs_sel_neus2)):
   mis_invs_sel_lay2.append(2)
   print(mis_invs_sel_labs2[indx], mis_invs_sel_neus2[indx], mis_invs_sel_sig2[indx], mis_invs_sel_supp2[indx])

invariants_ACT2 = get_all_invariants(basic_estimator2a)
#(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs2a, corr_invs_sel_neus2a, corr_invs_sel_sig2a, corr_invs_sel_supp2a) = describe_invariants_all_labels_col(invariants_ACT2)
(mis_invs_sel_labs3, mis_invs_sel_neus3, mis_invs_sel_sig3, mis_invs_sel_supp3, corr_invs_sel_labs3, corr_invs_sel_neus3, corr_invs_sel_sig3, corr_invs_sel_supp3) = describe_invariants_all_labels(invariants_ACT2)

print("\n CORRECT PATTERNS ACT 2:\n")
for indx in range(0, len(corr_invs_sel_neus3)):
   corr_invs_sel_lay2a.append(2)
   print(corr_invs_sel_labs3[indx], corr_invs_sel_neus3[indx], corr_invs_sel_sig3[indx], corr_invs_sel_supp3[indx])

print("\n INCORRECT PATTERNS ACT 2:\n")
for indx in range(0, len(mis_invs_sel_neus3)):
   mis_invs_sel_lay2a.append(2)
   print(mis_invs_sel_labs3[indx], mis_invs_sel_neus3[indx], mis_invs_sel_sig3[indx], mis_invs_sel_supp3[indx])

Obtained all paths


100%|██████████| 937/937 [00:00<00:00, 356177.53it/s]


 CORRECT PATTERNS VAL 1:

9 [11, 117, 43, 114, 41, 28, 116, 126, 114, 32, 69, 0, 54, 42, 122, 120, 28, 13, 65, 113, 114, 1, 82, 66, 112, 18, 72, 80, 106, 7, 1] ['<=', 3.39467716217041, '<=', 8.251644611358643, '>', 1.9294577836990356, '<=', 3.9608306884765625, '>', 0.5592330694198608, '>', 2.2061827182769775, '<=', 1.162885069847107, '<=', 3.762535572052002, '<=', 2.2799195051193237, '<=', 5.664603233337402, '<=', 1.917407512664795, '>', 3.5205564498901367, '<=', 8.098747730255127, '<=', 3.0489102602005005, '<=', -9.711438655853271, '<=', 3.8159372806549072, '<=', 9.273108005523682, '>', -9.07554817199707, '<=', -4.281953573226929, '<=', -2.5293995141983032, '>', -7.21074652671814, '<=', 6.236354351043701, '>', -3.394299864768982, '<=', -4.194207191467285, '>', -14.726406574249268, '<=', -1.135452151298523, '>', 1.3918424844741821, '<=', -4.226091384887695, '>', -5.114997386932373, '<=', -1.3256436586380005, '<=', 5.031131029129028] 712
9 [11, 117, 43, 114, 41, 12, 79, 96, 8, 126, 101

9
3 [11, 117, 43, 107, 49, 44, 93, 13, 54, 48, 101, 105] ['<=', 3.39467716217041, '<=', 8.251644611358643, '<=', 1.9294577836990356, '<=', 9.136926174163818, '<=', 2.7956079244613647, '<=', 3.092973828315735, '<=', 2.1124086380004883, '<=', -0.020313979126513004, '<=', 7.174140930175781, '<=', 5.874435901641846, '>', 4.926764011383057, '<=', 8.47739577293396] 6
3 [11, 13, 97, 54, 115, 32, 70, 49] ['>', 3.39467716217041, '<=', 2.2746496200561523, '>', -0.3762017488479614, '<=', 0.7059563398361206, '<=', 0.808624804019928, '<=', 4.693594217300415, '>', -1.9660648107528687, '<=', 3.0409834384918213] 6
3 [11, 117, 43, 107, 49, 69, 89, 60, 101, 38, 23, 104] ['<=', 3.39467716217041, '<=', 8.251644611358643, '<=', 1.9294577836990356, '<=', 9.136926174163818, '>', 2.7956079244613647, '<=', 0.012113329023122787, '<=', 5.2722086906433105, '>', -4.5517191886901855, '>', 1.1528421640396118, '<=', 3.6184439659118652, '>', -0.7742379903793335, '<=', -8.835254669189453] 5
3 [11, 117, 43, 114, 41, 28,

100%|██████████| 1922/1922 [00:00<00:00, 746638.17it/s]

...


 CORRECT PATTERNS VAL 2:

3 [11, 13, 97, 85, 115, 72, 38, 28, 54, 89, 64, 68, 105, 9, 4, 30, 61, 54, 121] ['>', 3.39467716217041, '<=', 2.2746496200561523, '>', 0.07813000679016113, '<=', 4.1136064529418945, '<=', 4.603406190872192, '>', 0.11424044519662857, '<=', 8.272120475769043, '<=', 3.1284674406051636, '>', 0.6984230279922485, '>', 2.337614059448242, '<=', 10.44788646697998, '<=', 10.319079399108887, '<=', 14.525535583496094, '<=', 4.887664079666138, '<=', 9.076398372650146, '>', 2.306199550628662, '<=', 11.020112991333008, '<=', 10.955713272094727, '<=', 6.7529027462005615] 908
3 [11, 13, 97, 85, 115, 72, 38, 28, 54, 89, 53, 54, 124, 12] ['>', 3.39467716217041, '<=', 2.2746496200561523, '>', 0.07813000679016113, '<=', 4.1136064529418945, '<=', 4.603406190872192, '>', 0.11424044519662857, '<=', 8.272120475769043, '<=', 3.1284674406051636, '>', 0.6984230279922485, '<=', 2.337614059448242, '<=', 1.9440388083457947, '>', 3.145810604095459, '<=', 5.723801612854004, '<=', 2.348825693

['>', 3.39467716217041, '>', 2.2746496200561523, '<=', 4.850768089294434, '>', 2.6442806720733643, '<=', 0.9984281659126282, '>', 1.929465115070343, '>', 2.715085029602051] 2
9 [11, 117, 43, 107, 49, 44, 93, 13, 54, 48, 126, 15, 64, 62, 116, 30] ['<=', 3.39467716217041, '<=', 8.251644611358643, '<=', 1.9294577836990356, '<=', 9.136926174163818, '<=', 2.7956079244613647, '<=', 3.092973828315735, '<=', 2.1124086380004883, '<=', 0.10921263694763184, '<=', 7.174140930175781, '<=', 5.874435901641846, '<=', 7.92810845375061, '<=', 3.8996111154556274, '<=', 7.387486457824707, '>', 3.6350677013397217, '<=', 0.7876976728439331, '<=', 3.552375376224518] 1
9 [11, 117, 43, 107, 49, 44, 93, 13, 54, 48, 48, 79, 54, 115, 12, 61] ['<=', 3.39467716217041, '<=', 8.251644611358643, '<=', 1.9294577836990356, '<=', 9.136926174163818, '<=', 2.7956079244613647, '<=', 3.092973828315735, '<=', 2.1124086380004883, '<=', 0.10921263694763184, '<=', 7.174140930175781, '>', 5.874435901641846, '<=', 12.7395195960998

100%|██████████| 1920/1920 [00:00<00:00, 804823.47it/s]


 CORRECT PATTERNS ACT 2:

5 [0, 28, 13, 15, 69, 23, 85, 11, 126, 49, 78] ['>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 356
5 [0, 28, 13, 15, 69, 23, 85, 11, 126, 49, 78, 62, 38] ['>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 70
5 [0, 28, 107, 68, 41, 116, 72, 13, 11, 121, 62] ['>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 35
5 [0, 28, 13, 15, 69, 23, 97, 85, 10, 116, 32, 89, 21] ['>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 34
5 [0, 28, 13, 15, 69, 23, 85, 11, 126, 97, 32] ['>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5] 28
5 [0, 28, 107, 68, 41, 101, 124, 10, 69, 93, 21] ['>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5

 [0, 28, 107, 68, 41, 101, 124, 32, 115, 97, 105, 13, 72, 49] ['>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 1
5 [0, 28, 107, 68, 41, 116, 72, 13, 11, 121, 62, 1, 64, 105] ['>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 1
5 [0, 28, 107, 68, 41, 116, 72, 69, 114, 119, 97, 76, 9, 27, 105, 95, 64, 32] ['>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 1
5 [0, 28, 107, 68, 41, 116, 72, 69, 62, 120, 42, 124, 48, 11, 13, 10] ['>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 1
5 [0, 28, 107, 68, 41, 116, 72, 69, 62, 120, 42, 15, 76, 97, 105] ['>', 0.5, 

In [ ]:
# INVOKE FOR EVERY CORRECT LABEL
def corr_match_pattern(layer_vals,suff,neuron_ids,neuron_sig):
   max_match_cnt = -1
   for ix in range(0,len(neuron_ids)):
     match_cnt = 0
     oper = -1
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         # print(v,vsig,val,oper)
         if (oper == 0):
          if (val <= vsig):
            match_cnt = match_cnt + 1
         else:
          if (val > vsig):
            match_cnt = match_cnt + 1
         oper = -1
         
     if (match_cnt > max_match_cnt):
       max_match_cnt = match_cnt

   return max_match_cnt  

def check_pattern_inter_lays(layer_vals1,layer_vals2,neuron_lay,neuron_ids,neuron_sig):
   
   labs = []
   supps = []
   found = True;
   oper = -1
   layer_vals1 = (layer_vals1).flatten()
   layer_vals2 = (layer_vals2).flatten()
   
   for ix in range(0,len(neuron_ids)):
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = 0
         if (neuron_lay[ix] == 1):
           val = layer_vals1[v]
         if (neuron_lay[ix] == 2):
           val = layer_vals2[v]
       
         #print("oper:",oper,"layer:",neuron_lay[ix],"pattern value:", v, "pattern sig:",vsig, "inp value:",val)

         if (oper == 0):
          if (val > vsig):
            found = False
          #  print('False')
            break
         else:
          if (val <= vsig):
            found = False
           # print('False')
            break
         oper = -1

     if (found == True):
       #break
       labs.append(ix) 
       #suppVal[ix] = suppVal[ix] + 1
       #supps.append(suppVal[ix])

   return labs
   

def check_pattern_inter(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):
   
   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   
   found = True;
   oper = -1
   layer_vals = (layer_vals).flatten()
   
   for ix in range(0,len(neuron_ids)):
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         #print(oper, v, vsig, val)
         if (oper == 0):
          if (val > vsig):
            found = False
          #  print('False')
            break
         else:
          if (val <= vsig):
            found = False
           # print('False')
            break
         oper = -1
     if (found == True):
       break 

   if (found == False):
      return -1
   else:
     return ix 

In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;
   #Integer v = null;
	#			Double vsig = null;
	#			String op = null;
	#			int oper = -1;
   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = neuron_ids[ind]
       vsig = neuron_sig[ind]
       val = layer_vals[v]
      # print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found 

In [ ]:
#### GUESSING THE IDEAL LABELS BASED ON PATTERNS

label0_tots = len(np.where(val_vec_labels1 == 0)[0])# and gen_act_labels == 0)[0])
label1_tots = len(np.where(val_vec_labels1 == 1)[0])# gen_act_labels == 1)[0])
label2_tots = len(np.where(val_vec_labels1 == 2)[0])#gen_act_labels == 2)[0])
label3_tots = len(np.where(val_vec_labels1 == 3)[0])
label4_tots = len(np.where(val_vec_labels1 == 4)[0])
label5_tots = len(np.where(val_vec_labels1 == 5)[0])
label6_tots = len(np.where(val_vec_labels1 == 6)[0])
label7_tots = len(np.where(val_vec_labels1 == 7)[0])
label8_tots = len(np.where(val_vec_labels1 == 8)[0])
label9_tots = len(np.where(val_vec_labels1 == 9)[0])

print(label0_tots,label1_tots,label2_tots,label3_tots,label4_tots,label5_tots)
print(label6_tots,label7_tots,label8_tots,label9_tots)
mis_tots = FAIL
print(mis_tots)

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

invs_sel_supp = []
invs_sel_neus = []
invs_sel_sig = []
invs_sel_labs = []
invs_sel_lay = []
invs_sel_suppVal = []
invs_sel_suppCVal = []
invs_sel_suppIVal = []


for ix in range (0, len(corr_invs_sel_neus)):
  if (corr_invs_sel_labs[ix] == 0):
    supp = float(corr_invs_sel_supp[ix]/label0_tots)
  if (corr_invs_sel_labs[ix] == 1):
    supp = float(corr_invs_sel_supp[ix]/label1_tots)
  if (corr_invs_sel_labs[ix] == 2):
    supp = float(corr_invs_sel_supp[ix]/label2_tots)
  if (corr_invs_sel_labs[ix] == 3):
    supp = float(corr_invs_sel_supp[ix]/label3_tots)
  if (corr_invs_sel_labs[ix] == 4):
    supp = float(corr_invs_sel_supp[ix]/label4_tots)
  if (corr_invs_sel_labs[ix] == 5):
    supp = float(corr_invs_sel_supp[ix]/label5_tots)
  if (corr_invs_sel_labs[ix] == 6):
    supp = float(corr_invs_sel_supp[ix]/label6_tots)
  if (corr_invs_sel_labs[ix] == 7):
    supp = float(corr_invs_sel_supp[ix]/label7_tots)
  if (corr_invs_sel_labs[ix] == 8):
    supp = float(corr_invs_sel_supp[ix]/label8_tots)  
  if (corr_invs_sel_labs[ix] == 9):
    supp = float(corr_invs_sel_supp[ix]/label9_tots)

  invs_sel_supp.append(supp*100.0)
  invs_sel_neus.append(corr_invs_sel_neus[ix])
  invs_sel_sig.append(corr_invs_sel_sig[ix])
  invs_sel_labs.append(corr_invs_sel_labs[ix])
  invs_sel_lay.append(corr_invs_sel_lay1[ix])
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)

for ix in range (0, len(corr_invs_sel_neus1)):
  if (corr_invs_sel_labs1[ix] == 0):
    supp = float(corr_invs_sel_supp1[ix]/label0_tots)
  if (corr_invs_sel_labs1[ix] == 1):
    supp = float(corr_invs_sel_supp1[ix]/label1_tots)
  if (corr_invs_sel_labs1[ix] == 2):
    supp = float(corr_invs_sel_supp1[ix]/label2_tots)
  if (corr_invs_sel_labs1[ix] == 3):
    supp = float(corr_invs_sel_supp1[ix]/label3_tots)
  if (corr_invs_sel_labs1[ix] == 4):
    supp = float(corr_invs_sel_supp1[ix]/label4_tots)
  if (corr_invs_sel_labs1[ix] == 5):
    supp = float(corr_invs_sel_supp1[ix]/label5_tots)
  if (corr_invs_sel_labs1[ix] == 6):
    supp = float(corr_invs_sel_supp1[ix]/label6_tots)
  if (corr_invs_sel_labs1[ix] == 7):
    supp = float(corr_invs_sel_supp1[ix]/label7_tots)
  if (corr_invs_sel_labs1[ix] == 8):
    supp = float(corr_invs_sel_supp1[ix]/label8_tots)  
  if (corr_invs_sel_labs1[ix] == 9):
    supp = float(corr_invs_sel_supp1[ix]/label9_tots)
  invs_sel_supp.append(supp*100.0)
  invs_sel_neus.append(corr_invs_sel_neus1[ix])
  invs_sel_sig.append(corr_invs_sel_sig1[ix])
  invs_sel_labs.append(corr_invs_sel_labs1[ix])
  invs_sel_lay.append(corr_invs_sel_lay1a[ix])
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)

for ix in range (0, len(corr_invs_sel_neus2)):
  if (corr_invs_sel_labs2[ix] == 0):
    supp = float(corr_invs_sel_supp2[ix]/label0_tots)
  if (corr_invs_sel_labs2[ix] == 1):
    supp = float(corr_invs_sel_supp2[ix]/label1_tots)
  if (corr_invs_sel_labs2[ix] == 2):
    supp = float(corr_invs_sel_supp2[ix]/label2_tots)
  if (corr_invs_sel_labs2[ix] == 3):
    supp = float(corr_invs_sel_supp2[ix]/label3_tots)
  if (corr_invs_sel_labs2[ix] == 4):
    supp = float(corr_invs_sel_supp2[ix]/label4_tots)
  if (corr_invs_sel_labs2[ix] == 5):
    supp = float(corr_invs_sel_supp2[ix]/label5_tots)
  if (corr_invs_sel_labs2[ix] == 6):
    supp = float(corr_invs_sel_supp2[ix]/label6_tots)
  if (corr_invs_sel_labs2[ix] == 7):
    supp = float(corr_invs_sel_supp2[ix]/label7_tots)
  if (corr_invs_sel_labs2[ix] == 8):
    supp = float(corr_invs_sel_supp2[ix]/label8_tots)  
  if (corr_invs_sel_labs2[ix] == 9):
    supp = float(corr_invs_sel_supp2[ix]/label9_tots)
  invs_sel_supp.append(supp*100.0)
  invs_sel_neus.append(corr_invs_sel_neus2[ix])
  invs_sel_sig.append(corr_invs_sel_sig2[ix])
  invs_sel_labs.append(corr_invs_sel_labs2[ix])
  invs_sel_lay.append(corr_invs_sel_lay2[ix])
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)

for ix in range (0, len(corr_invs_sel_neus3)):
  if (corr_invs_sel_labs3[ix] == 0):
    supp = float(corr_invs_sel_supp3[ix]/label0_tots)
  if (corr_invs_sel_labs3[ix] == 1):
    supp = float(corr_invs_sel_supp3[ix]/label1_tots)
  if (corr_invs_sel_labs3[ix] == 2):
    supp = float(corr_invs_sel_supp3[ix]/label2_tots)
  if (corr_invs_sel_labs3[ix] == 3):
    supp = float(corr_invs_sel_supp3[ix]/label3_tots)
  if (corr_invs_sel_labs3[ix] == 4):
    supp = float(corr_invs_sel_supp3[ix]/label4_tots)
  if (corr_invs_sel_labs3[ix] == 5):
    supp = float(corr_invs_sel_supp3[ix]/label5_tots)
  if (corr_invs_sel_labs3[ix] == 6):
    supp = float(corr_invs_sel_supp3[ix]/label6_tots)
  if (corr_invs_sel_labs3[ix] == 7):
    supp = float(corr_invs_sel_supp3[ix]/label7_tots)
  if (corr_invs_sel_labs3[ix] == 8):
    supp = float(corr_invs_sel_supp3[ix]/label8_tots)  
  if (corr_invs_sel_labs3[ix] == 9):
    supp = float(corr_invs_sel_supp3[ix]/label9_tots)
  invs_sel_supp.append(supp*100.0)
  invs_sel_neus.append(corr_invs_sel_neus3[ix])
  invs_sel_sig.append(corr_invs_sel_sig3[ix])
  invs_sel_labs.append(corr_invs_sel_labs3[ix])
  invs_sel_lay.append(corr_invs_sel_lay2a[ix]) 
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)


for ix in range (0, len(mis_invs_sel_neus)):
  invs_sel_supp.append(float(mis_invs_sel_supp[ix]/mis_tots)*100.0)
  invs_sel_neus.append(mis_invs_sel_neus[ix])
  invs_sel_sig.append(mis_invs_sel_sig[ix])
  invs_sel_labs.append(mis_invs_sel_labs[ix])
  invs_sel_lay.append(mis_invs_sel_lay1[ix])
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)

for ix in range (0, len(mis_invs_sel_neus1)):
  invs_sel_supp.append(float(mis_invs_sel_supp1[ix]/mis_tots)*100.0)
  #invs_sel_supp.append(mis_invs_sel_supp1[ix])
  invs_sel_neus.append(mis_invs_sel_neus1[ix])
  invs_sel_sig.append(mis_invs_sel_sig1[ix])
  invs_sel_labs.append(mis_invs_sel_labs1[ix])
  invs_sel_lay.append(mis_invs_sel_lay1a[ix])
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)

for ix in range (0, len(mis_invs_sel_neus2)):
  invs_sel_supp.append(float(mis_invs_sel_supp2[ix]/mis_tots)*100.0)
 # invs_sel_supp.append(mis_invs_sel_supp2[ix])
  invs_sel_neus.append(mis_invs_sel_neus2[ix])
  invs_sel_sig.append(mis_invs_sel_sig2[ix])
  invs_sel_labs.append(mis_invs_sel_labs2[ix])
  invs_sel_lay.append(mis_invs_sel_lay2[ix])
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)

for ix in range (0, len(mis_invs_sel_neus3)):
  invs_sel_supp.append(float(mis_invs_sel_supp3[ix]/mis_tots)*100.0)
#  invs_sel_supp.append(mis_invs_sel_supp3[ix])
  invs_sel_neus.append(mis_invs_sel_neus3[ix])
  invs_sel_sig.append(mis_invs_sel_sig3[ix])
  invs_sel_labs.append(mis_invs_sel_labs3[ix])
  invs_sel_lay.append(mis_invs_sel_lay2a[ix]) 
  invs_sel_suppCVal.append(0)
  invs_sel_suppIVal.append(0)
  invs_sel_suppVal.append(0)






for indx in range (0, len(invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(invs_sel_neus[indx])
          

for indx in range (0, len(invs_sel_sig)):
      Cneuron_sig.append(invs_sel_sig[indx])
      Cneuron_cl.append(invs_sel_labs[indx])
      Cneuron_lay.append(invs_sel_lay[indx])

#print("\n PATTERNS\n")
#for indx in range (0, len(Cneuron_cl)):
#  print(Cneuron_cl[indx],invs_sel_supp[indx],invs_sel_suppVal[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])

x = val_X
labels = val_Y
uncov = 0
sum_cor0 = 0
sum_incor0 = 0
sum_cor1 = 0
sum_incor1 = 0
sum_cor2 = 0
sum_incor2 = 0
sum_cor3 = 0
sum_incor3 = 0
sum_cor4 = 0
sum_incor4 = 0
sum_cor5 = 0
sum_incor5 = 0
sum_cor6 = 0
sum_incor6 = 0
sum_cor7 = 0
sum_incor7 = 0
sum_cor8 = 0
sum_incor8 = 0
sum_cor9 = 0
sum_incor9 = 0
sum_cor_mis = 0
sum_incor_mis = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = val_labels[inp_cnt]

    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]
    

    
    
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncov = uncov + 1
    cor = False
    cor1 = False
    for indx in range(0,len(matches)):
      index = matches[indx]
      lab = Cneuron_cl[index]  
      ##print("NEW LBL:", lab, ",IDEAL:", label)
      if (lab == 1000):
        if (actual != label):
          #print(actual, label, lab-10, "True")
          invs_sel_suppCVal[index] = invs_sel_suppCVal[index] + 1
          sum_cor_mis = sum_cor_mis+1
        else:
          #print(actual, label, lab-10, "False")
          invs_sel_suppIVal[index] = invs_sel_suppIVal[index] + 1
          sum_incor_mis = sum_incor_mis+1
        continue
      if (lab == label):
        invs_sel_suppCVal[index] = invs_sel_suppCVal[index] + 1
        cor = True
      else:
        invs_sel_suppIVal[index] = invs_sel_suppIVal[index] + 1
        cor = False
      if (label == 0):
        if (cor):
          sum_cor0 = sum_cor0+1
        else:
          sum_incor0 = sum_incor0+1
      if (label == 1):
        if (cor):
          sum_cor1 = sum_cor1+1
        else:
          sum_incor1 = sum_incor1+1
      if (label == 2):
        if (cor):
          sum_cor2 = sum_cor2+1
        else:
          sum_incor2 = sum_incor2+1
      if (label == 3):
        if (cor):
          sum_cor3 = sum_cor3+1
        else:
          sum_incor3 = sum_incor3+1
      if (label == 4):
        if (cor):
          sum_cor4 = sum_cor4+1
        else:
          sum_incor4 = sum_incor4+1
      if (label == 5):
        if (cor):
          sum_cor5 = sum_cor5+1
        else:
          sum_incor5 = sum_incor5+1
      if (label == 6):
        if (cor):
          sum_cor6 = sum_cor6+1
        else:
          sum_incor6 = sum_incor6+1
      if (label == 7):
        if (cor):
          sum_cor7 = sum_cor7+1
        else:
          sum_incor7 = sum_incor7+1
      if (label == 8):
        if (cor):
          sum_cor8 = sum_cor8+1
        else:
          sum_incor8 = sum_incor8+1
      if (label == 9):
        if (cor):
          sum_cor9 = sum_cor9+1
        else:
          sum_incor9 = sum_incor9+1








# invs_sel_suppVal1 = []
# Cneuron_cl1 = []
# invs_sel_supp1 = []
# Cneuron_lay1 = []
# C_neuron_ids1 = []
# Cneuron_sig1 = []
# print("\n PATTERNS AFTER\n")
# for indx in range (0, len(Cneuron_cl)):
#   invs_sel_suppVal[indx] = ((invs_sel_suppCVal[index]) / (invs_sel_suppCVal[index] + invs_sel_suppIVal[index]))*100.0
#   if (invs_sel_suppVal[indx] > 60.0):
#     Cneuron_cl1.append(Cneuron_cl[indx])
#     invs_sel_supp1.append(invs_sel_supp[indx])
#     Cneuron_lay1.append(Cneuron_lay[indx])
#     C_neuron_ids1.append(C_neuron_ids[indx])
#     Cneuron_sig1.append(Cneuron_sig[indx])
#     print(Cneuron_cl1[indx],invs_sel_supp1[indx],invs_sel_suppVal1[indx],Cneuron_lay1[indx],C_neuron_ids1[indx],Cneuron_sig1[indx])


# Cneuron_cl = Cneuron_cl1
# invs_sel_supp = invs_sel_supp1
# Cneuron_lay = Cneuron_lay1
# C_neuron_ids = C_neuron_ids1
# Cneuron_sig = Cneuron_sig1
# invs_sel_suppVal = invs_sel_suppVal1
print("VAL SET TOTAL RECALL:", float((len(val_X) - uncov)/len(val_X))*100.0)

print("VAL SET AVG PREC 0:", float(sum_cor0/(sum_cor0 + sum_incor0))*100.0)
print("VAL SET AVG PREC 1:", float(sum_cor1/(sum_cor1 + sum_incor1))*100.0)
print("VAL SET AVG PREC 2:", float(sum_cor2/(sum_cor2 + sum_incor2))*100.0)
print("VAL SET AVG PREC 3:", float(sum_cor3/(sum_cor3 + sum_incor3))*100.0)
print("VAL SET AVG PREC 4:", float(sum_cor4/(sum_cor4 + sum_incor4))*100.0)
print("VAL SET AVG PREC 5:", float(sum_cor5/(sum_cor5 + sum_incor5))*100.0)
print("VAL SET AVG PREC 6:", float(sum_cor6/(sum_cor6 + sum_incor6))*100.0)
print("VAL SET AVG PREC 7:", float(sum_cor7/(sum_cor7 + sum_incor7))*100.0)
print("VAL SET AVG PREC 8:", float(sum_cor8/(sum_cor8 + sum_incor8))*100.0)
print("VAL SET AVG PREC 9:", float(sum_cor9/(sum_cor9 + sum_incor9))*100.0)

print("VAL SET AVG PREC MIS:", float(sum_cor_mis/(sum_cor_mis + sum_incor_mis))*100.0)

1177 1329 1187 1285 1092 1050
1216 1218 1162 1139
796
VAL SET TOTAL RECALL: 100.0
VAL SET AVG PREC 0: 95.34177215189872
VAL SET AVG PREC 1: 96.72578444747613
VAL SET AVG PREC 2: 91.8918918918919
VAL SET AVG PREC 3: 89.14127423822714
VAL SET AVG PREC 4: 92.17199558985666
VAL SET AVG PREC 5: 89.31428571428572
VAL SET AVG PREC 6: 94.94505494505493
VAL SET AVG PREC 7: 89.92207792207793
VAL SET AVG PREC 8: 85.60397131825704
VAL SET AVG PREC 9: 90.03632589517385
VAL SET AVG PREC MIS: 8.349328214971209


In [ ]:
invs_sel_suppVal1 = []
Cneuron_cl1 = []
invs_sel_supp1 = []
Cneuron_lay1 = []
C_neuron_ids1 = []
Cneuron_sig1 = []
invs_sel_neus1 = []
invs_sel_sig1 = []
invs_sel_labs1 = []
invs_sel_lay1 = []


total_cov_0 = 0
total_cov_1 = 0
total_cov_2 = 0
total_cov_3 = 0
total_cov_4 = 0
total_cov_5 = 0
total_cov_6 = 0
total_cov_7 = 0
total_cov_8 = 0
total_cov_9 = 0

label0_tots = len(np.where(val_Y == 0)[0])# and gen_act_labels == 0)[0])
label1_tots = len(np.where(val_Y == 1)[0])# gen_act_labels == 1)[0])
label2_tots = len(np.where(val_Y == 2)[0])#gen_act_labels == 2)[0])
label3_tots = len(np.where(val_Y == 3)[0])
label4_tots = len(np.where(val_Y == 4)[0])
label5_tots = len(np.where(val_Y == 5)[0])
label6_tots = len(np.where(val_Y == 6)[0])
label7_tots = len(np.where(val_Y == 7)[0])
label8_tots = len(np.where(val_Y == 8)[0])
label9_tots = len(np.where(val_Y == 9)[0])

print(label0_tots,label1_tots,label2_tots,label3_tots,label4_tots,label5_tots)
print(label6_tots,label7_tots,label8_tots,label9_tots)

SUM_PREC = 0.0
print("\n PATTERNS for correct \n")
for indx in range (0, len(Cneuron_cl)):
  if (Cneuron_cl[indx] == 1000):
    continue
  if ((invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx]) == 0):
     invs_sel_suppVal[indx] = 0.0
     #total_uncov = total_uncov + 1
  else:
    invs_sel_suppVal[indx] = ((invs_sel_suppCVal[indx]) / (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx]))*100.0
  
 
  #print(invs_sel_suppVal[indx],invs_sel_suppCVal[indx],invs_sel_suppIVal[indx])
  #if (Cneuron_cl[indx] == 1000):
  #  print("class 1000:", invs_sel_suppVal[indx])
  if (invs_sel_suppVal[indx] >= 80.0):
    if (Cneuron_cl[indx] == 0):
      total_cov_0 = total_cov_0  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_0)
    if (Cneuron_cl[indx] == 1):
      total_cov_1 = total_cov_1  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_1)
    if (Cneuron_cl[indx] == 2):
      total_cov_2 = total_cov_2  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_2)
    if (Cneuron_cl[indx] == 3):
      total_cov_3 = total_cov_3  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_3)
    if (Cneuron_cl[indx] == 4):
      total_cov_4 = total_cov_4  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_4)
    if (Cneuron_cl[indx] == 5):
      total_cov_5 = total_cov_5  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_5)
    if (Cneuron_cl[indx] == 6):
      total_cov_6 = total_cov_6  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_6)
    if (Cneuron_cl[indx] == 7):
      total_cov_7 = total_cov_7  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_7)
    if (Cneuron_cl[indx] == 8):
      total_cov_8 = total_cov_8  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_8)
    if (Cneuron_cl[indx] == 9):
      total_cov_9 = total_cov_9  + (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx])
      #print(total_cov_9)
    Cneuron_cl1.append(Cneuron_cl[indx])
    invs_sel_supp1.append(invs_sel_supp[indx])
    Cneuron_lay1.append(Cneuron_lay[indx])
    C_neuron_ids1.append(C_neuron_ids[indx])
    Cneuron_sig1.append(Cneuron_sig[indx])
    invs_sel_suppVal1.append(invs_sel_suppVal[indx])
    invs_sel_neus1.append(invs_sel_neus[indx])
    invs_sel_sig1.append(invs_sel_sig[indx])
    invs_sel_labs1.append(invs_sel_labs[indx])
    invs_sel_lay1.append(invs_sel_lay[indx])
    indx1 = len(Cneuron_cl1) -1
    
    print(Cneuron_cl1[indx1],invs_sel_supp1[indx1],invs_sel_suppVal1[indx1],(invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx]),Cneuron_lay1[indx1],C_neuron_ids1[indx1],Cneuron_sig1[indx1])
   

print("PER LABEL VAL SET RECALL FOR PREC > 80%:")
print("lab 0:", float(total_cov_0/label0_tots)*100.0)
print("lab 1:", float(total_cov_1/label1_tots)*100.0)
print("lab 2:", float(total_cov_2/label2_tots)*100.0)
print("lab 3:", float(total_cov_3/label3_tots)*100.0)
print("lab 4:", float(total_cov_4/label4_tots)*100.0)
print("lab 5:", float(total_cov_5/label5_tots)*100.0)
print("lab 6:", float(total_cov_6/label6_tots)*100.0)
print("lab 7:", float(total_cov_7/label7_tots)*100.0)
print("lab 8:", float(total_cov_8/label8_tots)*100.0)
print("lab 9:", float(total_cov_9/label9_tots)*100.0)

# Cneuron_cl = Cneuron_cl1
# invs_sel_supp = invs_sel_supp1
# Cneuron_lay = Cneuron_lay1
# C_neuron_ids = C_neuron_ids1
# Cneuron_sig = Cneuron_sig1
# invs_sel_suppVal = invs_sel_suppVal1
# invs_sel_neus = invs_sel_neus1
# invs_sel_sig = invs_sel_sig1
# invs_sel_labs = invs_sel_labs1
# invs_sel_lay = invs_sel_lay1

502 569 507 478 475 471
471 522 487 517

 PATTERNS for correct 

9 62.510974539069366 98.78048780487805 328 1 [11, 117, 43, 114, 41, 28, 116, 126, 114, 32, 69, 0, 54, 42, 122, 120, 28, 13, 65, 113, 114, 1, 82, 66, 112, 18, 72, 80, 106, 7, 1] ['<=', 3.39467716217041, '<=', 8.251644611358643, '>', 1.9294577836990356, '<=', 3.9608306884765625, '>', 0.5592330694198608, '>', 2.2061827182769775, '<=', 1.162885069847107, '<=', 3.762535572052002, '<=', 2.2799195051193237, '<=', 5.664603233337402, '<=', 1.917407512664795, '>', 3.5205564498901367, '<=', 8.098747730255127, '<=', 3.0489102602005005, '<=', -9.711438655853271, '<=', 3.8159372806549072, '<=', 9.273108005523682, '>', -9.07554817199707, '<=', -4.281953573226929, '<=', -2.5293995141983032, '>', -7.21074652671814, '<=', 6.236354351043701, '>', -3.394299864768982, '<=', -4.194207191467285, '>', -14.726406574249268, '<=', -1.135452151298523, '>', 1.3918424844741821, '<=', -4.226091384887695, '>', -5.114997386932373, '<=', -1.32564365863800

In [ ]:
# invs_sel_suppVal1 = []
# Cneuron_cl1 = []
# invs_sel_supp1 = []
# Cneuron_lay1 = []
# C_neuron_ids1 = []
# Cneuron_sig1 = []
# invs_sel_neus1 = []
# invs_sel_sig1 = []
# invs_sel_labs1 = []
# invs_sel_lay1 = []

print("\n PATTERNS for incorrect \n")
for indx in range (0, len(Cneuron_cl)):
  if (Cneuron_cl[indx] < 10):
    continue
  if ((invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx]) == 0):
     invs_sel_suppVal[indx] = 0.0
  else:
    invs_sel_suppVal[indx] = ((invs_sel_suppCVal[indx]) / (invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx]))*100.0
  #print(invs_sel_suppVal[indx],invs_sel_suppCVal[indx],invs_sel_suppIVal[indx])
  #if (Cneuron_cl[indx] == 1000):
  #  print("class 1000:", invs_sel_suppVal[indx])
  if (invs_sel_suppVal[indx] >= 50.0):
    Cneuron_cl1.append(Cneuron_cl[indx])
    invs_sel_supp1.append(invs_sel_supp[indx])
    Cneuron_lay1.append(Cneuron_lay[indx])
    C_neuron_ids1.append(C_neuron_ids[indx])
    Cneuron_sig1.append(Cneuron_sig[indx])
    invs_sel_suppVal1.append(invs_sel_suppVal[indx])
    invs_sel_neus1.append(invs_sel_neus[indx])
    invs_sel_sig1.append(invs_sel_sig[indx])
    invs_sel_labs1.append(invs_sel_labs[indx])
    invs_sel_lay1.append(invs_sel_lay[indx])
    indx1 = len(Cneuron_cl1) -1
    print(Cneuron_cl1[indx1],invs_sel_supp1[indx1],invs_sel_suppVal1[indx1],(invs_sel_suppCVal[indx] + invs_sel_suppIVal[indx]),Cneuron_lay1[indx1],C_neuron_ids1[indx1],Cneuron_sig1[indx1])


Cneuron_cl = Cneuron_cl1
invs_sel_supp = invs_sel_supp1
Cneuron_lay = Cneuron_lay1
C_neuron_ids = C_neuron_ids1
Cneuron_sig = Cneuron_sig1
invs_sel_suppVal = invs_sel_suppVal1
invs_sel_neus = invs_sel_neus1
invs_sel_sig = invs_sel_sig1
invs_sel_labs = invs_sel_labs1
invs_sel_lay = invs_sel_lay1


 PATTERNS for incorrect 

1000 1.3819095477386936 50.0 4 1 [11, 117, 76, 27, 98, 93, 9, 96] ['<=', 3.39467716217041, '>', 8.251644611358643, '>', 2.550205945968628, '>', 1.8103091716766357, '>', 4.006288766860962, '>', 2.9236695766448975, '>', 2.219540596008301, '<=', 12.316730499267578]
1000 1.0050251256281406 50.0 2 1 [11, 13, 97, 54, 115, 53, 72, 23, 3, 51] ['>', 3.39467716217041, '<=', 2.2746496200561523, '>', -0.3762017488479614, '>', 0.7059563398361206, '<=', 4.603406190872192, '>', 3.58766508102417, '<=', 1.7227901220321655, '<=', 6.582728624343872, '<=', -2.6401039361953735, '>', -2.7281742095947266]
1000 0.8793969849246231 100.0 1 1 [11, 117, 76, 101, 21, 67, 62] ['<=', 3.39467716217041, '>', 8.251644611358643, '<=', 2.550205945968628, '<=', 0.9088682532310486, '<=', -0.1896507777273655, '>', -9.278894901275635, '>', -1.050360232591629]
1000 0.5025125628140703 100.0 1 1 [11, 13, 97, 54, 115, 53, 38, 85, 46] ['>', 3.39467716217041, '<=', 2.2746496200561523, '>', -0.37620174884

In [ ]:
print(len(invs_sel_supp),len(invs_sel_suppVal),len(invs_sel_neus),len(invs_sel_sig),len(invs_sel_labs),len(invs_sel_lay))
for ix in range (0, len(invs_sel_supp)-1):
    for iy in range(ix +1, len(invs_sel_supp)):
        if (invs_sel_supp[iy] > invs_sel_supp[ix]):
          temp = invs_sel_supp[iy]
          invs_sel_supp[iy] = invs_sel_supp[ix]
          invs_sel_supp[ix] = temp
          temp = invs_sel_suppVal[iy]
          invs_sel_suppVal[iy] = invs_sel_suppVal[ix]
          invs_sel_suppVal[ix] = temp
          temp = invs_sel_neus[iy]
          invs_sel_neus[iy] = invs_sel_neus[ix]
          invs_sel_neus[ix] = temp
          temp = invs_sel_sig[iy]
          invs_sel_sig[iy] = invs_sel_sig[ix]
          invs_sel_sig[ix] = temp
          temp = invs_sel_labs[iy]
          invs_sel_labs[iy] = invs_sel_labs[ix]
          invs_sel_labs[ix] = temp
          temp = invs_sel_lay[iy]
          invs_sel_lay[iy] = invs_sel_lay[ix]
          invs_sel_lay[ix] = temp

Cneuron_ids = []
C_neuron_ids = []
for indx in range (0, len(invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(invs_sel_neus[indx])
          
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []
for indx in range (0, len(invs_sel_sig)):
      Cneuron_sig.append(invs_sel_sig[indx])
      Cneuron_cl.append(invs_sel_labs[indx])
      Cneuron_lay.append(invs_sel_lay[indx])

print("\n PATTERNS SORTED BY TRN SUPP\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],invs_sel_supp[indx],invs_sel_suppVal[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])

1296 1296 1296 1296 1296 1296

 PATTERNS SORTED BY TRN SUPP

1 87.43416102332581 100.0 1 [11, 117, 76, 101, 21, 119, 98, 126, 118, 11, 86] ['<=', 3.39467716217041, '>', 8.251644611358643, '<=', 2.550205945968628, '<=', 0.9088682532310486, '>', -0.1896507777273655, '<=', 3.9502272605895996, '<=', 7.323296070098877, '>', -1.3715848922729492, '<=', -8.901604175567627, '>', -4.201511383056641, '<=', -2.4575753211975098]
1 86.45598194130926 100.0 2 [11, 117, 76, 101, 21, 119, 78, 42, 30, 19, 44, 33] ['<=', 3.39467716217041, '>', 8.251644611358643, '<=', 2.550205945968628, '<=', 0.9088682532310486, '>', 0.0005468390882015228, '<=', 3.9502272605895996, '<=', 6.284922122955322, '>', 0.028505053371191025, '<=', 9.005699634552002, '<=', 3.737175226211548, '<=', 8.082657814025879, '<=', 10.664632797241211]
0 78.07986406117247 99.48051948051948 1 [11, 117, 43, 107, 69, 28, 42, 121, 35, 35, 9, 33, 44, 12, 91, 44, 114] ['<=', 3.39467716217041, '<=', 8.251644611358643, '<=', 1.9294577836990356, '>', 

In [ ]:
print("TEST_BIASED SET:")

x = copy.deepcopy(test_biased_X)
labels = test_biased_Y
actuals = test_labels_biased
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = actuals[inp_cnt]

   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
      continue
    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

    for indx in range(0,len(matches)):
      index = matches[indx]
      fnd = False
      for ix in range(0,len(pat_indx1)):
        if (pat_indx1[ix] == index):
          fnd = True
          pat_cnt1[ix] = pat_cnt1[ix] + 1
          images_pat = pat_images1[ix]
          images_pat.append(inp_cnt)
          pat_images1[ix] = images_pat
          
      if (fnd == False):
        pat_indx1.append(index)
        pat_cnt1.append(1)
        pat_images1.append([inp_cnt])


print("TESTS UNCOVERED:", uncovered)

print("ONE INPUT -> MANY PAT")
for iy in range(0,len(pat_indx1)):
  index = pat_indx1[iy]
  lab = Cneuron_cl[index]
  trn_sup = invs_sel_supp[index]
  val_prec = invs_sel_suppVal[index]
  cor = 0

  for in_cnt in pat_images1[iy]:
    idl_lbl = labels[in_cnt]
    act_lbl = actuals[in_cnt]
    if (lab !=1000 and idl_lbl == lab):
      cor = cor + 1
    if (lab == 1000 and idl_lbl != act_lbl):
      cor = cor + 1
  print("LABEL:",lab,",Pattern:" , pat_indx1[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt1[iy], "COR%:", float(cor/pat_cnt1[iy])*100.0)


# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)


TEST_BIASED SET:
TESTS UNCOVERED: 26
ONE INPUT -> MANY PAT
LABEL: 9 ,Pattern: 15 ,TRN SUP: 62.510974539069366 ,VAL PREC: 98.78048780487805 ,CNT: 291 COR%: 100.0
LABEL: 9 ,Pattern: 19 ,TRN SUP: 56.71641791044776 ,VAL PREC: 99.35064935064936 ,CNT: 265 COR%: 100.0
LABEL: 9 ,Pattern: 86 ,TRN SUP: 5.267778753292362 ,VAL PREC: 90.47619047619048 ,CNT: 17 COR%: 100.0
LABEL: 9 ,Pattern: 87 ,TRN SUP: 5.267778753292362 ,VAL PREC: 90.47619047619048 ,CNT: 17 COR%: 100.0
LABEL: 9 ,Pattern: 209 ,TRN SUP: 2.28270412642669 ,VAL PREC: 100.0 ,CNT: 5 COR%: 100.0
LABEL: 9 ,Pattern: 211 ,TRN SUP: 2.28270412642669 ,VAL PREC: 100.0 ,CNT: 5 COR%: 100.0
LABEL: 0 ,Pattern: 2 ,TRN SUP: 78.07986406117247 ,VAL PREC: 99.48051948051948 ,CNT: 401 COR%: 99.75062344139651
LABEL: 0 ,Pattern: 3 ,TRN SUP: 77.90994052676295 ,VAL PREC: 99.2248062015504 ,CNT: 401 COR%: 99.75062344139651
LABEL: 0 ,Pattern: 17 ,TRN SUP: 58.53865760407817 ,VAL PREC: 99.42528735632183 ,CNT: 327 COR%: 99.69418960244649
LABEL: 0 ,Pattern: 18 ,TRN S

In [ ]:
print("TEST_UNBIASED SET:")

x = copy.deepcopy(test_unbiased_X)
labels = test_unbiased_Y
actuals = test_labels_unbiased
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = actuals[inp_cnt]

   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
      continue
    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

    for indx in range(0,len(matches)):
      index = matches[indx]
      fnd = False
      for ix in range(0,len(pat_indx1)):
        if (pat_indx1[ix] == index):
          fnd = True
          pat_cnt1[ix] = pat_cnt1[ix] + 1
          images_pat = pat_images1[ix]
          images_pat.append(inp_cnt)
          pat_images1[ix] = images_pat
          
      if (fnd == False):
        pat_indx1.append(index)
        pat_cnt1.append(1)
        pat_images1.append([inp_cnt])


print("TESTS UNCOVERED:", uncovered)

print("ONE INPUT -> MANY PAT")
for iy in range(0,len(pat_indx1)):
  index = pat_indx1[iy]
  lab = Cneuron_cl[index]
  trn_sup = invs_sel_supp[index]
  val_prec = invs_sel_suppVal[index]
  cor = 0

  for in_cnt in pat_images1[iy]:
    idl_lbl = labels[in_cnt]
    act_lbl = actuals[in_cnt]
    if (lab !=1000 and idl_lbl == lab):
      cor = cor + 1
    if (lab == 1000 and idl_lbl != act_lbl):
      cor = cor + 1
  print("LABEL:",lab,",Pattern:" , pat_indx1[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt1[iy], "COR%:", float(cor/pat_cnt1[iy])*100.0)


# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)


TEST_UNBIASED SET:
TESTS UNCOVERED: 40
ONE INPUT -> MANY PAT
LABEL: 1 ,Pattern: 0 ,TRN SUP: 87.43416102332581 ,VAL PREC: 100.0 ,CNT: 94 COR%: 100.0
LABEL: 1 ,Pattern: 1 ,TRN SUP: 86.45598194130926 ,VAL PREC: 100.0 ,CNT: 92 COR%: 100.0
LABEL: 1 ,Pattern: 22 ,TRN SUP: 43.566591422121896 ,VAL PREC: 99.61389961389962 ,CNT: 53 COR%: 100.0
LABEL: 1 ,Pattern: 23 ,TRN SUP: 43.566591422121896 ,VAL PREC: 99.61389961389962 ,CNT: 53 COR%: 100.0
LABEL: 4 ,Pattern: 4 ,TRN SUP: 76.46520146520146 ,VAL PREC: 96.97802197802197 ,CNT: 65 COR%: 100.0
LABEL: 4 ,Pattern: 6 ,TRN SUP: 74.08424908424908 ,VAL PREC: 97.55351681957187 ,CNT: 61 COR%: 100.0
LABEL: 4 ,Pattern: 152 ,TRN SUP: 3.296703296703297 ,VAL PREC: 85.71428571428571 ,CNT: 3 COR%: 100.0
LABEL: 4 ,Pattern: 153 ,TRN SUP: 3.296703296703297 ,VAL PREC: 85.71428571428571 ,CNT: 3 COR%: 100.0
LABEL: 3 ,Pattern: 5 ,TRN SUP: 75.09727626459144 ,VAL PREC: 98.03370786516854 ,CNT: 53 COR%: 100.0
LABEL: 3 ,Pattern: 9 ,TRN SUP: 70.6614785992218 ,VAL PREC: 98.4984

In [ ]:
print("TEST_PASS SET:")

x = copy.deepcopy(test_pass_X)
labels = test_pass_Y
actuals = test_labels_pass
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = test_labels_pass[inp_cnt]

   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
      continue
    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

    for indx in range(0,len(matches)):
      index = matches[indx]
      fnd = False
      for ix in range(0,len(pat_indx1)):
        if (pat_indx1[ix] == index):
          fnd = True
          pat_cnt1[ix] = pat_cnt1[ix] + 1
          images_pat = pat_images1[ix]
          images_pat.append(inp_cnt)
          pat_images1[ix] = images_pat
          
      if (fnd == False):
        pat_indx1.append(index)
        pat_cnt1.append(1)
        pat_images1.append([inp_cnt])


print("TESTS UNCOVERED:", uncovered)

uncovered1 = 0
print("ONE INPUT -> MANY PAT")
for iy in range(0,len(pat_indx1)):
  index = pat_indx1[iy]
  lab = Cneuron_cl[index]
  trn_sup = invs_sel_supp[index]
  val_prec = invs_sel_suppVal[index]
  cor = 0
  if (len(pat_images1[iy]) == 0):
    uncovered1 = uncovered1 + 1
  for in_cnt in pat_images1[iy]:
    idl_lbl = labels[in_cnt]
    act_lbl = actuals[in_cnt]
    if (lab !=1000 and idl_lbl == lab):
      cor = cor + 1
    if (lab == 1000 and idl_lbl != act_lbl):
      cor = cor + 1
  print("LABEL:",lab,",Pattern:" , pat_indx1[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt1[iy], "COR%:", float(cor/pat_cnt1[iy])*100.0)

print("OBLIGATIONS UNCOVERED:", uncovered1)
# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)


TEST_PASS SET:
TESTS UNCOVERED: 8
ONE INPUT -> MANY PAT
LABEL: 7 ,Pattern: 10 ,TRN SUP: 70.11494252873564 ,VAL PREC: 99.40652818991099 ,CNT: 14 COR%: 100.0
LABEL: 7 ,Pattern: 13 ,TRN SUP: 65.76354679802957 ,VAL PREC: 99.38271604938271 ,CNT: 15 COR%: 100.0
LABEL: 7 ,Pattern: 34 ,TRN SUP: 27.175697865353037 ,VAL PREC: 100.0 ,CNT: 6 COR%: 100.0
LABEL: 7 ,Pattern: 35 ,TRN SUP: 27.175697865353037 ,VAL PREC: 100.0 ,CNT: 6 COR%: 100.0
LABEL: 1 ,Pattern: 0 ,TRN SUP: 87.43416102332581 ,VAL PREC: 100.0 ,CNT: 23 COR%: 100.0
LABEL: 1 ,Pattern: 1 ,TRN SUP: 86.45598194130926 ,VAL PREC: 100.0 ,CNT: 23 COR%: 100.0
LABEL: 1 ,Pattern: 40 ,TRN SUP: 14.973664409330322 ,VAL PREC: 98.3739837398374 ,CNT: 5 COR%: 100.0
LABEL: 1 ,Pattern: 41 ,TRN SUP: 14.973664409330322 ,VAL PREC: 98.3739837398374 ,CNT: 5 COR%: 100.0
LABEL: 8 ,Pattern: 24 ,TRN SUP: 35.197934595524956 ,VAL PREC: 98.77300613496932 ,CNT: 10 COR%: 100.0
LABEL: 8 ,Pattern: 25 ,TRN SUP: 35.111876075731494 ,VAL PREC: 99.3975903614458 ,CNT: 9 COR%: 10

In [ ]:
print("TEST_PASS_FAIL SET:")

x = copy.deepcopy(test_pass_fail_X)
labels = test_pass_fail_Y
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = test_labels_pass_fail[inp_cnt]
   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
      continue
    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

    for indx in range(0,len(matches)):
      index = matches[indx]
      fnd = False
      for ix in range(0,len(pat_indx1)):
        if (pat_indx1[ix] == index):
          fnd = True
          pat_cnt1[ix] = pat_cnt1[ix] + 1
          images_pat = pat_images1[ix]
          images_pat.append(inp_cnt)
          pat_images1[ix] = images_pat
          
      if (fnd == False):
        pat_indx1.append(index)
        pat_cnt1.append(1)
        pat_images1.append([inp_cnt])


print("TESTS UNCOVERED:", uncovered)
uncovered1 = 0
print("ONE INPUT -> MANY PAT")
for iy in range(0,len(pat_indx1)):
  index = pat_indx1[iy]
  lab = Cneuron_cl[index]
  trn_sup = invs_sel_supp[index]
  val_prec = invs_sel_suppVal[index]
  cor = 0
  if (len(pat_images1[iy]) == 0):
    uncovered1 = uncovered1 + 1
  for in_cnt in pat_images1[iy]:
    idl_lbl = labels[in_cnt]
    act_lbl = actuals[in_cnt]
    if (lab != 1000 and idl_lbl == lab):
      cor = cor + 1
    if (lab == 1000 and idl_lbl != actual):
      cor = cor + 1
  print("LABEL:",lab,",Pattern:" , pat_indx1[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt1[iy], "COR%:", float(cor/pat_cnt1[iy])*100.0)

print("OBLIGATIONS UNCOVERED:", ((uncovered1)/len(pat_indx1))*100)
# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)


TEST_PASS_FAIL SET:
TESTS UNCOVERED: 33
ONE INPUT -> MANY PAT
LABEL: 5 ,Pattern: 14 ,TRN SUP: 65.33333333333333 ,VAL PREC: 99.34426229508196 ,CNT: 15 COR%: 93.33333333333333
LABEL: 5 ,Pattern: 16 ,TRN SUP: 61.71428571428571 ,VAL PREC: 100.0 ,CNT: 13 COR%: 100.0
LABEL: 5 ,Pattern: 26 ,TRN SUP: 33.904761904761905 ,VAL PREC: 100.0 ,CNT: 5 COR%: 100.0
LABEL: 5 ,Pattern: 27 ,TRN SUP: 33.904761904761905 ,VAL PREC: 100.0 ,CNT: 5 COR%: 100.0
LABEL: 1 ,Pattern: 0 ,TRN SUP: 87.43416102332581 ,VAL PREC: 100.0 ,CNT: 10 COR%: 100.0
LABEL: 1 ,Pattern: 1 ,TRN SUP: 86.45598194130926 ,VAL PREC: 100.0 ,CNT: 10 COR%: 100.0
LABEL: 1 ,Pattern: 96 ,TRN SUP: 5.041384499623777 ,VAL PREC: 100.0 ,CNT: 2 COR%: 100.0
LABEL: 1 ,Pattern: 97 ,TRN SUP: 5.041384499623777 ,VAL PREC: 100.0 ,CNT: 2 COR%: 100.0
LABEL: 5 ,Pattern: 149 ,TRN SUP: 3.3333333333333335 ,VAL PREC: 91.66666666666666 ,CNT: 2 COR%: 100.0
LABEL: 5 ,Pattern: 150 ,TRN SUP: 3.3333333333333335 ,VAL PREC: 91.66666666666666 ,CNT: 2 COR%: 100.0
LABEL: 7 ,Pa

In [ ]:
print("FULL TEST SET:")

x = copy.deepcopy(test_X)
labels = test_Y
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = test_labels[inp_cnt]
   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
      continue
    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

    for indx in range(0,len(matches)):
      index = matches[indx]
      fnd = False
      for ix in range(0,len(pat_indx1)):
        if (pat_indx1[ix] == index):
          fnd = True
          pat_cnt1[ix] = pat_cnt1[ix] + 1
          images_pat = pat_images1[ix]
          images_pat.append(inp_cnt)
          pat_images1[ix] = images_pat
          
      if (fnd == False):
        pat_indx1.append(index)
        pat_cnt1.append(1)
        pat_images1.append([inp_cnt])


print("TESTS UNCOVERED:", uncovered)

print("ONE INPUT -> MANY PAT")
for iy in range(0,len(pat_indx1)):
  index = pat_indx1[iy]
  lab = Cneuron_cl[index]
  trn_sup = invs_sel_supp[index]
  val_prec = invs_sel_suppVal[index]
  cor = 0
  
  for in_cnt in pat_images1[iy]:
    idl_lbl = labels[in_cnt]
    actual = test_labels[in_cnt]
    if (lab != 1000 and idl_lbl == lab):
      cor = cor + 1
    if (lab == 1000 and idl_lbl != actual):
      cor = cor + 1
  print("LABEL:",lab,",Pattern:" , pat_indx1[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt1[iy], "COR%:", float(cor/pat_cnt1[iy])*100.0)

# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)


FULL TEST SET:
TESTS UNCOVERED: 211
ONE INPUT -> MANY PAT
LABEL: 7 ,Pattern: 10 ,TRN SUP: 70.11494252873564 ,VAL PREC: 99.40652818991099 ,CNT: 318 COR%: 98.74213836477988
LABEL: 7 ,Pattern: 13 ,TRN SUP: 65.76354679802957 ,VAL PREC: 99.38271604938271 ,CNT: 298 COR%: 99.32885906040269
LABEL: 7 ,Pattern: 34 ,TRN SUP: 27.175697865353037 ,VAL PREC: 100.0 ,CNT: 112 COR%: 98.21428571428571
LABEL: 7 ,Pattern: 35 ,TRN SUP: 27.175697865353037 ,VAL PREC: 100.0 ,CNT: 112 COR%: 98.21428571428571
LABEL: 6 ,Pattern: 467 ,TRN SUP: 0.8223684210526315 ,VAL PREC: 88.88888888888889 ,CNT: 14 COR%: 92.85714285714286
LABEL: 6 ,Pattern: 468 ,TRN SUP: 0.8223684210526315 ,VAL PREC: 88.88888888888889 ,CNT: 14 COR%: 92.85714285714286
LABEL: 1 ,Pattern: 0 ,TRN SUP: 87.43416102332581 ,VAL PREC: 100.0 ,CNT: 467 COR%: 100.0
LABEL: 1 ,Pattern: 1 ,TRN SUP: 86.45598194130926 ,VAL PREC: 100.0 ,CNT: 464 COR%: 100.0
LABEL: 1 ,Pattern: 40 ,TRN SUP: 14.973664409330322 ,VAL PREC: 98.3739837398374 ,CNT: 126 COR%: 99.2063492063

In [ ]:
print("FULL TEST ADV SET:")

x = copy.deepcopy(test_adv_X)
labels = test_adv_Y
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = test_labels_adv[inp_cnt]
   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
      continue
    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

    for indx in range(0,len(matches)):
      index = matches[indx]
      fnd = False
      for ix in range(0,len(pat_indx1)):
        if (pat_indx1[ix] == index):
          fnd = True
          pat_cnt1[ix] = pat_cnt1[ix] + 1
          images_pat = pat_images1[ix]
          images_pat.append(inp_cnt)
          pat_images1[ix] = images_pat
          
      if (fnd == False):
        pat_indx1.append(index)
        pat_cnt1.append(1)
        pat_images1.append([inp_cnt])


print("UNCOVERED:", uncovered)

print("ONE INPUT -> MANY PAT")
for iy in range(0,len(pat_indx1)):
  index = pat_indx1[iy]
  lab = Cneuron_cl[index]
  trn_sup = invs_sel_supp[index]
  val_prec = invs_sel_suppVal[index]
  cor = 0
  for in_cnt in pat_images1[iy]:
    idl_lbl = labels[in_cnt]
    actual = test_labels[in_cnt]
    if (lab != 1000 and idl_lbl == lab):
      cor = cor + 1
    if (lab == 1000 and idl_lbl != actual):
      cor = cor + 1
  print("LABEL:",lab,",Pattern:" , pat_indx1[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt1[iy], "COR%:", float(cor/pat_cnt1[iy])*100.0)

# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)


FULL TEST ADV SET:
UNCOVERED: 594
ONE INPUT -> MANY PAT
LABEL: 6 ,Pattern: 467 ,TRN SUP: 0.8223684210526315 ,VAL PREC: 88.88888888888889 ,CNT: 10 COR%: 60.0
LABEL: 6 ,Pattern: 468 ,TRN SUP: 0.8223684210526315 ,VAL PREC: 88.88888888888889 ,CNT: 10 COR%: 60.0
LABEL: 1 ,Pattern: 0 ,TRN SUP: 87.43416102332581 ,VAL PREC: 100.0 ,CNT: 219 COR%: 100.0
LABEL: 1 ,Pattern: 1 ,TRN SUP: 86.45598194130926 ,VAL PREC: 100.0 ,CNT: 235 COR%: 93.19148936170212
LABEL: 1 ,Pattern: 40 ,TRN SUP: 14.973664409330322 ,VAL PREC: 98.3739837398374 ,CNT: 70 COR%: 84.28571428571429
LABEL: 1 ,Pattern: 41 ,TRN SUP: 14.973664409330322 ,VAL PREC: 98.3739837398374 ,CNT: 70 COR%: 84.28571428571429
LABEL: 4 ,Pattern: 4 ,TRN SUP: 76.46520146520146 ,VAL PREC: 96.97802197802197 ,CNT: 199 COR%: 99.49748743718592
LABEL: 4 ,Pattern: 6 ,TRN SUP: 74.08424908424908 ,VAL PREC: 97.55351681957187 ,CNT: 247 COR%: 78.13765182186235
LABEL: 4 ,Pattern: 198 ,TRN SUP: 2.380952380952381 ,VAL PREC: 100.0 ,CNT: 12 COR%: 75.0
LABEL: 4 ,Pattern:

In [ ]:
print("ORACLE: FULL TEST SET:")

x = copy.deepcopy(test_X)
labels = test_Y
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []

tot_0 = np.where((labels == 0))[0]
print("TOT 0:", len(tot_0))
tot_1 = np.where((labels == 1))[0]
tot_2 = np.where((labels == 2))[0]
tot_3 = np.where((labels == 3))[0]
tot_4 = np.where((labels == 4))[0]
tot_5 = np.where((labels == 5))[0]
tot_6 = np.where((labels == 6))[0]
tot_7 = np.where((labels == 7))[0]
tot_8 = np.where((labels == 8))[0]
tot_9 = np.where((labels == 9))[0]
cor_0 = 0
incor_0 = 0
cor_1 = 0
incor_1 = 0
cor_2 = 0
incor_2 = 0
cor_3 = 0
incor_3 = 0
cor_4 = 0
incor_4 = 0
cor_5 = 0
incor_5 = 0
cor_6 = 0
incor_6 = 0
cor_7 = 0
incor_7 = 0
cor_8 = 0
incor_8 = 0
cor_9 = 0
incor_9 = 0

cor_cor = 0
cor_incor = 0
mis_cor = 0
mis_incor = 0
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = test_labels[inp_cnt]
   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
    else:
      index = matches[0]
      lab = Cneuron_cl[index]
      if (lab != 1000):
        if (label == 0):
          if (lab == label):
            cor_0 = cor_0 +1
          else:
            incor_0 = incor_0 +1
      
        if (label == 1):
          if (lab == label):
            cor_1 = cor_1 +1
          else:
            incor_1 = incor_1 +1
     
        if (label == 2):
          if (lab == label):
            cor_2 = cor_2 +1
          else:
            incor_2 = incor_2 +1

        if (label == 3):
          if (lab == label):
            cor_3 = cor_3 +1
          else:
            incor_3 = incor_3 +1
      
        if (label == 4):
          if (lab == label):
            cor_4 = cor_4 +1
          else:
            incor_4 = incor_4 +1
     
        if (label == 5):
          if (lab == label):
            cor_5 = cor_5 +1
          else:
            incor_5 = incor_5 +1

        if (label == 6):
          if (lab == label):
            cor_6 = cor_6 +1
          else:
            incor_6 = incor_6 +1
      
        if (label == 7):
          if (lab == label):
            cor_7 = cor_7 +1
          else:
            incor_7 = incor_7 +1
     
        if (label == 8):
          if (lab == label):
            cor_8 = cor_8 +1
          else:
            incor_8 = incor_8 +1

        if (label == 9):
          if (lab == label):
            cor_9 = cor_9 +1
          else:
            incor_9 = incor_9 +1

        if (actual == label):
          cor_cor = cor_cor + 1
        else:
          cor_incor = cor_incor + 1

      else:
        if (actual != label):
          mis_cor = mis_cor + 1
        else:
          mis_incor = mis_incor + 1

    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

print("TESTS UNCOVERED:", uncovered)
print(cor_cor,cor_incor,mis_cor,mis_incor )
print("PRECISION PREDICTION OF CORRECT MODEL BEHAVIOR:", float(cor_cor/ (cor_cor +cor_incor))*100.0)
print("RECALL OF CORRECT CLASSIFICATION PATTERNS:", float((cor_cor +cor_incor)/len(x))*100.0)
print("PRECISION PREDICTION OF INCCORRECT MODEL BEHAVIOR:", float(mis_cor/ (mis_cor +mis_incor))*100.0)
print("RECALL OF INCORRECT CLASSIFICATION PATTERNS:", float((mis_cor +mis_incor)/len(x))*100.0)
print("TOTAL PRECISION of model behavior pred:", float((cor_cor + mis_cor)/(cor_incor+mis_incor+cor_cor + mis_cor))*100.0)

print("TOTAL RECALL:", float((cor_cor + mis_cor+cor_incor+mis_incor)/(len(x))*100.0))


print("PRECISION PREDICTION OF GROUND TRUTH BEHAVIOR:")
print("LABEL 0:", float(cor_0/ (cor_0 + incor_0))*100.0)
print("LABEL 1:", float(cor_1/ (cor_1 + incor_1))*100.0)
print("LABEL 2:", float(cor_2/ (cor_2 + incor_2))*100.0)
print("LABEL 3:", float(cor_3/ (cor_3 + incor_3))*100.0)
print("LABEL 4:", float(cor_4/ (cor_4 + incor_4))*100.0)
print("LABEL 5:", float(cor_5/ (cor_5 + incor_5))*100.0)
print("LABEL 6:", float(cor_6/ (cor_6 + incor_6))*100.0)
print("LABEL 7:", float(cor_7/ (cor_7 + incor_7))*100.0)
print("LABEL 8:", float(cor_8/ (cor_8 + incor_8))*100.0)
print("LABEL 9:", float(cor_9/ (cor_9 + incor_9))*100.0)

print("RECALL OF CORRECT CLASSIFICATON PATTERNS FOR DIFFERENT LABELS:")
print("LABEL 0:", float((cor_0 + incor_0)/len(tot_0))*100.0)
print("LABEL 1:", float((cor_1 + incor_1)/len(tot_1))*100.0)
print("LABEL 2:", float((cor_2 + incor_2)/len(tot_2))*100.0)
print("LABEL 3:", float((cor_3 + incor_3)/len(tot_3))*100.0)
print("LABEL 4:", float((cor_4 + incor_4)/len(tot_4))*100.0)
print("LABEL 5:", float((cor_5 + incor_5)/len(tot_5))*100.0)
print("LABEL 6:", float((cor_6 + incor_6)/len(tot_6))*100.0)
print("LABEL 7:", float((cor_7 + incor_7)/len(tot_7))*100.0)
print("LABEL 8:", float((cor_8 + incor_8)/len(tot_8))*100.0)
print("LABEL 9:", float((cor_9 + incor_9)/len(tot_9))*100.0)


print("TOTAL PRECISION of ground truth pred:", float((cor_0 + cor_1 + cor_2 +cor_3 + cor_4 + cor_5 + cor_6 +cor_7 + cor_8 + cor_9)/(cor_0 + cor_1 + cor_2 +cor_3 + cor_4 + cor_5 + cor_6 +cor_7 + cor_8 + cor_9 + incor_0 + incor_1 + incor_2 + incor_3 + incor_4 + incor_5 + incor_6 + incor_7 + incor_8 + incor_9))*100.0)

print("TOTAL RECALL:", float((cor_0 + cor_1 + cor_2 +cor_3 + cor_4 + cor_5 + cor_6 +cor_7 + cor_8 + cor_9 + incor_0 + incor_1 + incor_2 + incor_3 + incor_4 + incor_5 + incor_6 + incor_7 + incor_8 + incor_9)/(len(tot_0)+len(tot_1)+len(tot_2)+len(tot_3)+len(tot_4)+len(tot_5)+len(tot_6)+len(tot_7)+len(tot_8)+len(tot_9)) )*100.0)



# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)


ORACLE: FULL TEST SET:
TOT 0: 478
TESTS UNCOVERED: 211
4689 89 2 10
PRECISION PREDICTION OF CORRECT MODEL BEHAVIOR: 98.13729593972373
RECALL OF CORRECT CLASSIFICATION PATTERNS: 95.54089182163568
PRECISION PREDICTION OF INCCORRECT MODEL BEHAVIOR: 16.666666666666664
RECALL OF INCORRECT CLASSIFICATION PATTERNS: 0.23995200959808036
TOTAL PRECISION of model behavior pred: 97.93319415448852
TOTAL RECALL: 95.78084383123375
PRECISION PREDICTION OF GROUND TRUTH BEHAVIOR:
LABEL 0: 98.5200845665962
LABEL 1: 98.74551971326166
LABEL 2: 93.37349397590361
LABEL 3: 93.54838709677419
LABEL 4: 95.9349593495935
LABEL 5: 95.2020202020202
LABEL 6: 96.41350210970464
LABEL 7: 96.88796680497926
LABEL 8: 94.5823927765237
LABEL 9: 93.77682403433477
RECALL OF CORRECT CLASSIFICATON PATTERNS FOR DIFFERENT LABELS:
LABEL 0: 98.9539748953975
LABEL 1: 98.58657243816255
LABEL 2: 94.85714285714286
LABEL 3: 93.23308270676691
LABEL 4: 97.0414201183432
LABEL 5: 94.06175771971496
LABEL 6: 97.3305954825462
LABEL 7: 95.256916

In [ ]:
print("ADV SET ORACLE:")

x = copy.deepcopy(test_X_adv_2)
labels = testFULL_Y
pat_indx = []
pat_cnt = []
pat_images = []
pat_indx1 = []
pat_cnt1 = []
pat_images1 = []

tot_0 = len(np.where((labels == 0))[0])
tot_1 = len(np.where((labels == 1))[0])
tot_2 = len(np.where((labels == 2))[0])
tot_3 = len(np.where((labels == 3))[0])
tot_4 = len(np.where((labels == 4))[0])
tot_5 = len(np.where((labels == 5))[0])
tot_6 = len(np.where((labels == 6))[0])
tot_7 = len(np.where((labels == 7))[0])
tot_8 = len(np.where((labels == 8))[0])
tot_9 = len(np.where((labels == 9))[0])

cor_0 = 0
incor_0 = 0
cor_1 = 0
incor_1 = 0
cor_2 = 0
incor_2 = 0
cor_3 = 0
incor_3 = 0
cor_4 = 0
incor_4 = 0
cor_5 = 0
incor_5 = 0
cor_6 = 0
incor_6 = 0
cor_7 = 0
incor_7 = 0
cor_8 = 0
incor_8 = 0
cor_9 = 0
incor_9 = 0

cor_cor = 0
cor_incor = 0
mis_cor = 0
mis_incor = 0
uncovered = 0
for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    image = x[inp_cnt]
    label = labels[inp_cnt]
    actual = test_labels_adv_2[inp_cnt]
   
    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=image

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy]))

    img_mod = np.expand_dims(img,axis=0)


    inp = image 
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []
    
    inp = image 
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    
   
    matches = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    if (len(matches) == 0):
      uncovered = uncovered + 1
    else:
      index = matches[0]
      lab = Cneuron_cl[index]
      if (lab != 1000):
        if (label == 0):
          if (lab == label):
            cor_0 = cor_0 +1
          else:
            incor_0 = incor_0 +1
      
        if (label == 1):
          if (lab == label):
            cor_1 = cor_1 +1
          else:
            incor_1 = incor_1 +1
     
        if (label == 2):
          if (lab == label):
            cor_2 = cor_2 +1
          else:
            incor_2 = incor_2 +1

        if (label == 3):
          if (lab == label):
            cor_3 = cor_3 +1
          else:
            incor_3 = incor_3 +1
      
        if (label == 4):
          if (lab == label):
            cor_4 = cor_4 +1
          else:
            incor_4 = incor_4 +1
     
        if (label == 5):
          if (lab == label):
            cor_5 = cor_5 +1
          else:
            incor_5 = incor_5 +1

        if (label == 6):
          if (lab == label):
            cor_6 = cor_6 +1
          else:
            incor_6 = incor_6 +1
      
        if (label == 7):
          if (lab == label):
            cor_7 = cor_7 +1
          else:
            incor_7 = incor_7 +1
     
        if (label == 8):
          if (lab == label):
            cor_8 = cor_8 +1
          else:
            incor_8 = incor_8 +1

        if (label == 9):
          if (lab == label):
            cor_9 = cor_9 +1
          else:
            incor_9 = incor_9 +1

        if (actual == label):
          cor_cor = cor_cor + 1
        else:
          cor_incor = cor_incor + 1

      else:
        if (actual != label):
          mis_cor = mis_cor + 1
        else:
          mis_incor = mis_incor + 1

    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

print("TESTS UNCOVERED:", uncovered)
print("PRECISION PREDICTION OF CORRECT MODEL BEHAVIOR:", float(cor_cor/ (cor_cor +cor_incor))*100.0)
print("RECALL OF CORRECT CLASSIFICATION PATTERNS:", float((cor_cor +cor_incor)/len(x))*100.0)
print("PRECISION PREDICTION OF INCCORRECT MODEL BEHAVIOR:", float(mis_cor/ (mis_cor +mis_incor))*100.0)
print("RECALL OF INCORRECT CLASSIFICATION PATTERNS:", float((mis_cor +mis_incor)/len(x))*100.0)
print("TOTAL PRECISION of model behavior pred:", float((cor_cor + mis_cor)/(cor_incor+mis_incor+cor_cor + mis_cor))*100.0)
print("TOTAL RECALL:", float((cor_cor + mis_cor+cor_incor+mis_incor)/(len(x))*100.0))


print("PRECISION PREDICTION OF GROUND TRUTH BEHAVIOR:")
print("LABEL 0:", float(cor_0/ (cor_0 + incor_0))*100.0)
print("LABEL 1:", float(cor_1/ (cor_1 + incor_1))*100.0)
print("LABEL 2:", float(cor_2/ (cor_2 + incor_2))*100.0)
print("LABEL 3:", float(cor_3/ (cor_3 + incor_3))*100.0)
print("LABEL 4:", float(cor_4/ (cor_4 + incor_4))*100.0)
print("LABEL 5:", float(cor_5/ (cor_5 + incor_5))*100.0)
print("LABEL 6:", float(cor_6/ (cor_6 + incor_6))*100.0)
print("LABEL 7:", float(cor_7/ (cor_7 + incor_7))*100.0)
print("LABEL 8:", float(cor_8/ (cor_8 + incor_8))*100.0)
print("LABEL 9:", float(cor_9/ (cor_9 + incor_9))*100.0)

print("RECALL OF CORRECT CLASSIFICATON PATTERNS FOR DIFFERENT LABELS:")
print("LABEL 0:", float((cor_0 + incor_0)/tot_0)*100.0)
print("LABEL 1:", float((cor_1 + incor_1)/tot_1)*100.0)
print("LABEL 2:", float((cor_2 + incor_2)/tot_2)*100.0)
print("LABEL 3:", float((cor_3 + incor_3)/tot_3)*100.0)
print("LABEL 4:", float((cor_4 + incor_4)/tot_4)*100.0)
print("LABEL 5:", float((cor_5 + incor_5)/tot_5)*100.0)
print("LABEL 6:", float((cor_6 + incor_6)/tot_6)*100.0)
print("LABEL 7:", float((cor_7 + incor_7)/tot_7)*100.0)
print("LABEL 8:", float((cor_8 + incor_8)/tot_8)*100.0)
print("LABEL 9:", float((cor_9 + incor_9)/tot_9)*100.0)

      

print("TOTAL PRECISION of ground truth pred:", float((cor_0 + cor_1 + cor_2 +cor_3 + cor_4 + cor_5 + cor_6 +cor_7 + cor_8 + cor_9)/(cor_0 + cor_1 + cor_2 +cor_3 + cor_4 + cor_5 + cor_6 +cor_7 + cor_8 + cor_9 + incor_0 + incor_1 + incor_2 + incor_3 + incor_4 + incor_5 + incor_6 + incor_7 + incor_8 + incor_9))*100.0)

print("TOTAL RECALL:", float((cor_0 + cor_1 + cor_2 +cor_3 + cor_4 + cor_5 + cor_6 +cor_7 + cor_8 + cor_9 + incor_0 + incor_1 + incor_2 + incor_3 + incor_4 + incor_5 + incor_6 + incor_7 + incor_8 + incor_9)/((tot_0)+(tot_1)+(tot_2)+(tot_3)+(tot_4)+(tot_5)+(tot_6)+(tot_7)+(tot_8)+(tot_9)) )*100.0)
  

    #for indx in range(0,len(matches)):
    # if (len(matches) > 0):
    #   indx = 0
    #   index = matches[indx]
    #   fnd = False
    #   for ix in range(0,len(pat_indx)):
    #     if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
    #       break
    #   if (fnd == False):
    #     pat_indx.append(index)
    #     pat_cnt.append(1)
    #     pat_images.append([inp_cnt])
    # else:
    #   uncovered = uncovered + 1

    #matches1 = check_pattern_inter_lays(layer_vals1,layer_vals2,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    # index = matches[0]
    # fnd = False
    # for ix in range(0,len(pat_indx)):
    #   if (pat_indx[ix] == index):
    #       fnd = True
    #       pat_cnt[ix] = pat_cnt[ix] + 1
    #       images_pat = pat_images[ix]
    #       images_pat.append(inp_cnt)
    #       pat_images[ix] = images_pat
          
    # if (fnd == False):
    #   pat_indx.append(index)
    #   pat_cnt.append(1)
    #   pat_images.append([inp_cnt])

# print("ONE INPUT -> ONE PAT")
# for iy in range(0,len(pat_indx)):
#   index = pat_indx[iy]
#   lab = Cneuron_cl[index]
#   trn_sup = invs_sel_supp[index]
#   val_prec = invs_sel_suppVal[index]
#   cor = 0
#   for in_cnt in pat_images[iy]:
#     idl_lbl = labels[in_cnt]
#     if (idl_lbl == lab):
#       cor = cor + 1
#     if (lab == 1000 and idl_lbl != actual):
#       cor = cor + 1
#   print("LABEL:",lab,",Pattern:" , pat_indx[iy], ",TRN SUP:", trn_sup, ",VAL PREC:", val_prec ,",CNT:", pat_cnt[iy], "COR%:", float(cor/pat_cnt[iy])*100.0)
